In [1]:
import sys

In [2]:
sys.executable

'/Users/christanasescu/opt/anaconda3/bin/python3'

In [3]:
sys.path

['/Users/christanasescu',
 '/Users/christanasescu/opt/anaconda3/lib/python37.zip',
 '/Users/christanasescu/opt/anaconda3/lib/python3.7',
 '/Users/christanasescu/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/christanasescu/opt/anaconda3/lib/python3.7/site-packages',
 '/Users/christanasescu/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/christanasescu/.ipython']

In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.tokenize import sent_tokenize, word_tokenize
import os
#The OS module in Python provides a way of using operating system dependent functionality. 
#The functions that the OS module provides allows you to interface with the underlying operating system 
#that Python is running on – be that Windows, Mac or Linux.

from os import listdir
from os.path import isfile, join

# Gensim
import gensim
import gensim.corpora as corpora
from gensim import models, corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

ModuleNotFoundError: No module named 'pyLDAvis'

In [6]:
import nltk as nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christanasescu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
stopwords = nltk.corpus.stopwords.words('stop_words_poetry.txt')

stopwords.append('...')
stopwords.append("'d")
stopwords.append('...')
stopwords.append("&")
stopwords.append("upon")
stopwords.append("also")
stopwords.append("hath")
stopwords.append("must")
stopwords.append("therefore")
stopwords.append("doth")
stopwords.append("could")
stopwords.append("would")
stopwords.append("another")
stopwords.append("much")
stopwords.append("give")
stopwords.append("like")
stopwords.append("since")
stopwords.append("many")
stopwords.append("without")
stopwords.append("first")
stopwords.append("though")
stopwords.append("well")
stopwords.append("often")
stopwords.append("great")
stopwords.append("either")
stopwords.append("even")
stopwords.append("shall")
stopwords.append("they")
stopwords.append("what")
stopwords.append("their")
stopwords.append("more")
stopwords.append("there")
stopwords.append("your")
stopwords.append("them")

In [8]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = _pre_clean(tokens)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in stopwords]
    #tokens = [get_lemma(token) for token in tokens]
    return tokens

In [9]:
pwd

'/Users/christanasescu'

In [10]:
import os

In [11]:
#cd fastText_multilingual-master

In [12]:

HOME = os.getcwd()

In [13]:
def _pre_clean(list_of_text):
        '''
        preliminary cleaning of the text
        - remove new line character i.e. \n or \r
        - remove tabs i.e. \t
        - remove extra spaces
        '''
        cleaned_list = []
        for text in list_of_text:
            # print("original:", text)
            text = text.replace('\\n', ' ')
            text = text.replace('\\r', ' ')
            text = text.replace('\\t', ' ')
            pattern = re.compile(r'\s+')
            text = re.sub(pattern, ' ', text)
            text = text.strip()
            text = text.lower()
            # check for empty strings
            if text != '' and text is not None:
                cleaned_list.append(text)

        return cleaned_list

In [16]:
TEXTS_DIR = HOME + "/rushton_&_funkhouser_for_margento_trans/"

filelabels = {}

texts_data = []

files = [f for f in os.listdir(TEXTS_DIR) if os.path.isfile(os.path.join(TEXTS_DIR, f))]

import string
from string import punctuation

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

tokens_total = []

count = -1
 
os.chdir(TEXTS_DIR)
    
for f in files:
    #os.chdir(TEXTS_DIR)
    with open(f, "r", encoding='utf-8', errors = 'ignore') as openf:
        tokens = []
        count = count + 1
        filelabels[count] = os.path.basename(openf.name)
        for line in openf:
            sent_text = nltk.sent_tokenize(line)
            for sentence in sent_text:
                tokens1 = tokenize(sentence)
                tokens1 = [item.translate(remove_punct_map)
                      for item in tokens1]
                #filter_object = filter(lambda x: x != "", tokens1)
                tokens1 = [x for x in tokens1 if x!= ""]
                for token in tokens1:
                    tokens.append(token)
                    tokens_total.append(token)
                #if random.random() > .99:
                #print(tokens)
    #print(tokens_total)
    texts_data.append(tokens)

print(filelabels)

{0: 'rushton_&_funkhouser.txt'}


In [13]:

len(tokens_total)

7720

In [17]:
tokens_total = [x for x in tokens_total if x not in stopwords]

In [15]:

len(tokens_total)

7720

In [19]:
from collections import Counter

Count_total = Counter(tokens_total)

In [20]:

print(Count_total)

Counter({'work': 65, 'verse': 62, 'ovid': 52, 'poetry': 42, 'time': 42, 'other': 39, 'poet': 39, 'poets': 32, 'digital': 31, 'audio': 31, 'lines': 30, 'poem': 30, 'writing': 28, 'when': 26, 'media': 25, 'write': 25, 'beat': 23, 'line': 23, 'marlowe': 22, 'sound': 21, 'both': 21, 'might': 21, 'dryden': 20, 'course': 19, 'thought': 19, 'half': 19, 'said': 19, 'research': 18, 'three': 18, 'voice': 18, 'used': 18, 'recordings': 18, 'project': 18, 'make': 18, 'thanks': 18, 'best': 17, 'production': 17, 'online': 17, 'magazine': 16, 'made': 16, 'form': 16, 'where': 15, 'every': 15, 'poems': 15, 'years': 14, 'better': 14, 'humanities': 13, 'others': 13, 'life': 13, 'making': 13, 'each': 13, 'perhaps': 13, 'know': 13, 'plan': 13, 'between': 13, 'pennsound': 13, 'translation': 13, 'rhythm': 13, 'alter': 13, 'working': 12, 'literary': 12, 'voices': 12, 'technology': 12, 'least': 12, 'only': 12, 'order': 12, 'material': 12, 'came': 12, 'type': 12, 'before': 12, 'part': 12, 'types': 11, 'creative'

In [21]:

for i in range(1):
    print(len(texts_data[i]))

7899


In [12]:

for i in range(1):
    texts_data[i] = [x for x in texts_data[i] if x not in stopwords]

In [17]:

print(len(texts_data[0]))

7720


In [18]:

dictionary = corpora.Dictionary(texts_data)

In [19]:

corpus = [dictionary.doc2bow(text) for text in texts_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [26]:

def compute_coherence_values(dictionary, corpus, texts, limit, start = 2, step = 1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus = corpus, num_topics = num_topics, id2word = dictionary, random_state = 100, update_every = 1, passes = 15, alpha = 'auto', per_word_topics = True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model = model, texts = texts, dictionary = dictionary, coherence = 'c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [27]:

model_list, coherence_values = compute_coherence_values(dictionary = dictionary, corpus = corpus, texts = texts_data, start=2, limit=10, step=1)

In [28]:

limit=10
start=2
step=1
x = range(start, limit, step)

for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.2937
Num Topics = 3  has Coherence Value of 0.2833
Num Topics = 4  has Coherence Value of 0.2925
Num Topics = 5  has Coherence Value of 0.2971
Num Topics = 6  has Coherence Value of 0.3023
Num Topics = 7  has Coherence Value of 0.2945
Num Topics = 8  has Coherence Value of 0.3039
Num Topics = 9  has Coherence Value of 0.2978


In [29]:

NUM_TOPICS = 8
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, random_state=100, update_every=1, passes=15, alpha='auto', per_word_topics=True)

In [34]:

topics = ldamodel.print_topics(num_words=46)

In [35]:

pprint(topics)

[(0,
  '0.000*"verse" + 0.000*"ovid" + 0.000*"work" + 0.000*"other" + 0.000*"poet" '
  '+ 0.000*"poetry" + 0.000*"audio" + 0.000*"time" + 0.000*"poem" + '
  '0.000*"poets" + 0.000*"marlowe" + 0.000*"digital" + 0.000*"lines" + '
  '0.000*"media" + 0.000*"both" + 0.000*"when" + 0.000*"said" + 0.000*"beat" + '
  '0.000*"might" + 0.000*"write" + 0.000*"recordings" + 0.000*"made" + '
  '0.000*"writing" + 0.000*"project" + 0.000*"thanks" + 0.000*"three" + '
  '0.000*"dryden" + 0.000*"half" + 0.000*"production" + 0.000*"thought" + '
  '0.000*"research" + 0.000*"sound" + 0.000*"line" + 0.000*"course" + '
  '0.000*"voice" + 0.000*"making" + 0.000*"online" + 0.000*"used" + '
  '0.000*"type" + 0.000*"humanities" + 0.000*"magazine" + 0.000*"form" + '
  '0.000*"every" + 0.000*"plan" + 0.000*"perhaps" + 0.000*"again"'),
 (1,
  '0.000*"ovid" + 0.000*"work" + 0.000*"verse" + 0.000*"poetry" + 0.000*"time" '
  '+ 0.000*"poem" + 0.000*"lines" + 0.000*"audio" + 0.000*"other" + '
  '0.000*"poets" + 0.000*"

In [20]:

NUM_TOPICS = 2
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, random_state=100, update_every=1, passes=15, alpha='auto', per_word_topics=True)

In [16]:

topics = ldamodel.print_topics(num_words=46)

In [38]:

pprint(topics)

[(0,
  '0.007*"work" + 0.006*"verse" + 0.005*"ovid" + 0.004*"poetry" + 0.004*"time" '
  '+ 0.004*"poet" + 0.004*"other" + 0.003*"poets" + 0.003*"audio" + '
  '0.003*"digital" + 0.003*"poem" + 0.003*"lines" + 0.003*"writing" + '
  '0.003*"when" + 0.003*"media" + 0.003*"write" + 0.002*"beat" + 0.002*"line" '
  '+ 0.002*"marlowe" + 0.002*"both" + 0.002*"might" + 0.002*"sound" + '
  '0.002*"dryden" + 0.002*"said" + 0.002*"half" + 0.002*"thought" + '
  '0.002*"course" + 0.002*"project" + 0.002*"voice" + 0.002*"three" + '
  '0.002*"recordings" + 0.002*"thanks" + 0.002*"research" + 0.002*"used" + '
  '0.002*"make" + 0.002*"production" + 0.002*"online" + 0.002*"best" + '
  '0.002*"form" + 0.002*"made" + 0.002*"magazine" + 0.002*"every" + '
  '0.002*"poems" + 0.002*"where" + 0.001*"years" + 0.001*"better"'),
 (1,
  '0.000*"work" + 0.000*"verse" + 0.000*"ovid" + 0.000*"time" + 0.000*"poetry" '
  '+ 0.000*"lines" + 0.000*"poets" + 0.000*"poem" + 0.000*"digital" + '
  '0.000*"audio" + 0.000*"other

In [21]:

l0 = list(ldamodel.show_topic(0, topn = 9000))

In [22]:
l1 = list(ldamodel.show_topic(1, topn = 9000))

In [19]:

len(l0)

3647

In [25]:
len(l1)

3647

In [43]:

for i in range(3593):
    if l0[i][0]== "hosts":
            print(i)

1618


In [44]:

for i in range(3593):
    if l1[i][0]== "hosts":
            print(i)

3219


In [45]:

for i in range(1597, 1639):
    print(l0[i])

('curve', 0.00015470575)
('sporadically', 0.00015470575)
('doubled', 0.00015470573)
('expressed', 0.00015470572)
('circulated', 0.00015470572)
('receiver', 0.0001547057)
('lots', 0.00015470569)
('include', 0.00015470567)
('entail', 0.00015470567)
('why—it', 0.00015470565)
('international', 0.00015470563)
('discouraging', 0.00015470563)
('drummers', 0.0001547056)
('dispute', 0.00015470559)
('jgheaburi', 0.00015470559)
('imagination', 0.00015470557)
('loan', 0.00015470554)
('proliferation', 0.00015470553)
('succeeded', 0.00015470551)
('phrase', 0.00015470551)
('heralding', 0.00015470544)
('hosts', 0.00015470535)
('feel', 0.00015470531)
('cities', 0.00015470524)
('draw', 0.00015470517)
('torn', 0.00015470517)
('doua', 0.00015470512)
('rightly', 0.0001547051)
('copied', 0.00015470505)
('apă', 0.00015470505)
('hotel', 0.00015470503)
('opinion—mere', 0.00015470502)
('apprenticeship', 0.00015470495)
('courbet', 0.00015470493)
('circuits', 0.00015470492)
('majorminor—', 0.00015470492)
('messy'

In [46]:

for i in range(3198, 3240):
    print(l1[i])

('countries', 0.00027738238)
('oral', 0.00027738232)
('dobruja—', 0.00027738215)
('schimb', 0.00027738186)
('typical', 0.00027738183)
('messy', 0.00027738156)
('circuits', 0.00027738154)
('majorminor—', 0.00027738154)
('conviction', 0.00027738148)
('courbet', 0.00027738145)
('apprenticeship', 0.00027738136)
('opinion—mere', 0.00027738107)
('hotel', 0.0002773809)
('apă', 0.0002773808)
('copied', 0.00027738066)
('rightly', 0.0002773804)
('draw', 0.00027738008)
('doua', 0.00027738005)
('torn', 0.00027737988)
('cities', 0.00027737988)
('feel', 0.00027737935)
('hosts', 0.0002773792)
('heralding', 0.0002773786)
('succeeded', 0.00027737825)
('phrase', 0.00027737816)
('imagination', 0.00027737813)
('proliferation', 0.00027737799)
('loan', 0.00027737799)
('drummers', 0.00027737799)
('dispute', 0.00027737796)
('jgheaburi', 0.00027737775)
('why—it', 0.00027737767)
('international', 0.00027737764)
('discouraging', 0.00027737764)
('entail', 0.00027737746)
('lots', 0.00027737746)
('include', 0.00027

In [47]:

for i in range(3593):
    if l0[i][0]== "order":
            print(i)

65


In [48]:

for i in range(3785):
    if l1[i][0]== "order":
            print(i)

68


IndexError: list index out of range

In [49]:

for i in range(44, 86):
    print(l0[i])

('years', 0.0014956939)
('better', 0.0014956937)
('making', 0.0013931122)
('others', 0.0013929555)
('humanities', 0.0013929214)
('plan', 0.0013928963)
('pennsound', 0.0013927348)
('each', 0.0013927148)
('alter', 0.0013926955)
('perhaps', 0.0013926775)
('translation', 0.0013925313)
('know', 0.0013924867)
('between', 0.0013924098)
('life', 0.0013924078)
('rhythm', 0.0013923275)
('only', 0.0012896862)
('type', 0.0012896707)
('literary', 0.0012896046)
('part', 0.001289567)
('voices', 0.001289509)
('technology', 0.0012894431)
('order', 0.0012894189)
('came', 0.0012893794)
('material', 0.0012893633)
('working', 0.0012893245)
('least', 0.0012892204)
('before', 0.0012890969)
('over', 0.0011865739)
('verses', 0.0011865603)
('times', 0.0011865592)
('again', 0.0011864719)
('creative', 0.0011864519)
('electronic', 0.0011863944)
('later', 0.0011863864)
('types', 0.0011863599)
('word', 0.0011862848)
('wine', 0.0011862806)
('take', 0.0011862704)
('things', 0.0011862076)
('something', 0.0011861869)
('

In [50]:

for i in range(47, 89):
    print(l1[i])

('said', 0.00028704328)
('every', 0.00028687992)
('most', 0.0002867616)
('know', 0.00028664275)
('here', 0.00028654104)
('least', 0.0002864969)
('translation', 0.0002864019)
('form', 0.0002861089)
('down', 0.0002859939)
('production', 0.00028594976)
('working', 0.00028593963)
('good', 0.00028581006)
('being', 0.00028574013)
('material', 0.00028573253)
('sense', 0.00028568166)
('came', 0.00028564624)
('perhaps', 0.0002856192)
('written', 0.00028555116)
('alter', 0.0002855225)
('text', 0.00028546122)
('find', 0.00028543544)
('order', 0.00028543395)
('each', 0.00028541687)
('pennsound', 0.0002853118)
('technology', 0.00028530572)
('something', 0.00028510767)
('because', 0.0002850112)
('things', 0.0002849951)
('voices', 0.00028495255)
('based', 0.00028491768)
('practice', 0.0002846967)
('early', 0.00028468)
('take', 0.00028466227)
('someone', 0.0002846588)
('part', 0.00028464056)
('wine', 0.00028460834)
('word', 0.00028458514)
('plan', 0.00028444768)
('literary', 0.00028444218)
('archives'

In [51]:

for i in range(3593):
    if l0[i][0]== "resonates":
            print(i)

2332


In [52]:
for i in range(3593):
    if l1[i][0]== "resonates":
            print(i)

1581


In [53]:

for i in range(2311, 2353):
    print(l0[i])

('housed', 0.0001546892)
('archaeology', 0.0001546892)
('woodside', 0.00015468919)
('contracted', 0.00015468914)
('chipping', 0.00015468907)
('fivebeat', 0.00015468906)
('errors', 0.00015468903)
('selective', 0.00015468903)
('drum', 0.00015468901)
('appeared', 0.00015468901)
('catchsome', 0.00015468901)
('laps', 0.000154689)
('interjects', 0.00015468898)
('fields', 0.00015468897)
('clock', 0.00015468895)
('website', 0.00015468894)
('hufnagel', 0.00015468891)
('hates', 0.00015468891)
('godhungry', 0.0001546889)
('preverbs', 0.00015468888)
('translate', 0.00015468887)
('resonates', 0.00015468887)
('garden—dryden', 0.00015468885)
('microcommunity', 0.00015468882)
('trece', 0.00015468881)
('numerous', 0.0001546888)
('freedom', 0.00015468878)
('ageless', 0.00015468878)
('originalele', 0.00015468874)
('tatters', 0.00015468874)
('venit', 0.00015468869)
('outdated', 0.00015468866)
('poetrybeam', 0.00015468866)
('219', 0.00015468863)
('difficult', 0.00015468862)
('8track', 0.00015468862)
('frie

In [54]:

for i in range(1560, 1601):
    print(l1[i])

('strange', 0.000277583)
('hierarchies', 0.0002775828)
('related', 0.00027758259)
('intersection', 0.00027758256)
('contaminate', 0.0002775821)
('blue', 0.000277582)
('retrospective', 0.000277582)
('sections', 0.0002775818)
('movement', 0.00027758154)
('rudimentary', 0.00027758142)
('xxxiii', 0.000277581)
('kills', 0.00027758084)
('bottles—drinks', 0.0002775808)
('crude', 0.0002775808)
('santana', 0.0002775808)
('displayed', 0.0002775807)
('solos—yes', 0.00027758066)
('likely', 0.00027757988)
('andrews', 0.0002775798)
('selection', 0.00027757976)
('appears', 0.00027757973)
('contented', 0.00027757973)
('cleft', 0.00027757918)
('appreciation', 0.00027757906)
('usefully', 0.00027757886)
('designing', 0.00027757883)
('regression', 0.00027757883)
('reaching', 0.0002775788)
('travel', 0.00027757874)
('inventive', 0.0002775781)
('challenged', 0.00027757804)
('linebased', 0.00027757796)
('wine—he', 0.00027757746)
('fear', 0.00027757735)
('rods', 0.00027757708)
('keeping', 0.00027757662)
('vie

In [31]:

for i in range(3593):
    if l0[i][0]== "scanning":
            print(i)

1861


In [32]:

for i in range(3593):
    if l1[i][0]== "scanning":
            print(i)

2976


In [33]:

for i in range(1840, 1882):
    print(l0[i])

('richness', 0.00015469902)
('drowned', 0.00015469902)
('powers', 0.00015469897)
('be—and', 0.00015469897)
('foreign', 0.00015469894)
('doctoral', 0.00015469892)
('penchant', 0.00015469892)
('daunting', 0.00015469885)
('constructed', 0.00015469885)
('detritus', 0.00015469882)
('dissecting', 0.0001546988)
('suits', 0.00015469879)
('admitting', 0.00015469879)
('speak', 0.00015469873)
('believes', 0.00015469872)
('project—a', 0.0001546987)
('contribution', 0.00015469868)
('rent', 0.00015469866)
('tutun', 0.00015469866)
('ești', 0.00015469866)
('easytofollow', 0.00015469863)
('scanning', 0.00015469857)
('humanity', 0.00015469847)
('shed', 0.00015469847)
('repetition—', 0.00015469841)
('pole', 0.00015469841)
('delivery', 0.00015469838)
('perform', 0.00015469831)
('extended', 0.0001546983)
('except', 0.0001546983)
('nightline', 0.00015469825)
('propel', 0.00015469825)
('undiscovered', 0.00015469825)
('conversion', 0.00015469822)
('toilet', 0.00015469822)
('ending', 0.00015469817)
('sellby', 

In [34]:

for i in range(2955, 2999):
    print(l1[i])

('literally', 0.00027741797)
('referred', 0.00027741794)
('podcasts', 0.00027741774)
('advances', 0.00027741768)
('occurrence', 0.00027741762)
('entails', 0.00027741762)
('sellby', 0.0002774174)
('ending', 0.00027741736)
('toilet', 0.00027741716)
('conversion', 0.00027741707)
('undiscovered', 0.00027741698)
('propel', 0.00027741687)
('nightline', 0.00027741678)
('extended', 0.00027741672)
('perform', 0.00027741664)
('except', 0.00027741655)
('repetition—', 0.00027741608)
('pole', 0.00027741602)
('delivery', 0.000277416)
('humanity', 0.00027741582)
('shed', 0.0002774158)
('scanning', 0.00027741533)
('rent', 0.00027741477)
('easytofollow', 0.00027741477)
('ești', 0.00027741474)
('contribution', 0.00027741469)
('tutun', 0.00027741466)
('project—a', 0.00027741454)
('believes', 0.00027741442)
('speak', 0.0002774142)
('admitting', 0.00027741404)
('suits', 0.00027741404)
('dissecting', 0.00027741393)
('detritus', 0.0002774138)
('constructed', 0.00027741375)
('daunting', 0.0002774136)
('pencha

In [37]:

for i in range(3593):
    if l0[i][0]== "symmetry":
            print(i)

3466


In [38]:
for i in range(3593):
    if l1[i][0]== "symmetry":
            print(i)

1371


In [39]:

for i in range(3444, 3488):
    print(l0[i])

('described', 0.00015465746)
('equivalent', 0.0001546574)
('lotto', 0.0001546574)
('mainstream', 0.00015465733)
('relocating', 0.000154657)
('suit', 0.00015465694)
('burnt', 0.00015465694)
('baggaley', 0.00015465687)
('brilliant', 0.0001546568)
('selfless', 0.00015465668)
('rambles', 0.00015465643)
('propulsiveness', 0.00015465643)
('afternoons', 0.00015465636)
('amendments', 0.00015465633)
('presents', 0.00015465623)
('rhythmrhyme', 0.00015465615)
('misadventure', 0.00015465604)
('proceeds', 0.00015465604)
('zola', 0.00015465597)
('exorbitant', 0.00015465594)
('—was', 0.00015465592)
('locate', 0.00015465579)
('symmetry', 0.00015465573)
('briefly', 0.00015465572)
('lake', 0.00015465565)
('it—say', 0.0001546555)
('around—', 0.00015465547)
('converse', 0.00015465531)
('tickets', 0.0001546553)
('web—might', 0.00015465527)
('elaborate', 0.00015465527)
('brian', 0.00015465516)
('pool', 0.00015465515)
('clear', 0.00015465506)
('tied', 0.00015465503)
('gtrrunes', 0.00015465502)
('imagined', 0

In [40]:

for i in range(1350, 1392):
    print(l1[i])

('onward', 0.00027764987)
('cunning', 0.0002776495)
('solution', 0.00027764926)
('7000', 0.00027764894)
('frutti', 0.00027764792)
('anybody', 0.0002776478)
('fluent', 0.00027764754)
('imagined', 0.0002776468)
('clear', 0.00027764632)
('gtrrunes', 0.00027764632)
('tied', 0.0002776463)
('pool', 0.0002776458)
('brian', 0.00027764568)
('web—might', 0.00027764527)
('converse', 0.00027764504)
('tickets', 0.00027764498)
('elaborate', 0.00027764498)
('around—', 0.00027764417)
('it—say', 0.00027764356)
('lake', 0.00027764324)
('briefly', 0.0002776427)
('symmetry', 0.00027764248)
('locate', 0.0002776424)
('—was', 0.0002776417)
('zola', 0.00027764155)
('exorbitant', 0.0002776415)
('proceeds', 0.0002776411)
('misadventure', 0.00027764105)
('rhythmrhyme', 0.0002776405)
('presents', 0.0002776399)
('amendments', 0.0002776393)
('afternoons', 0.00027763928)
('rambles', 0.00027763908)
('propulsiveness', 0.0002776389)
('selfless', 0.00027763756)
('brilliant', 0.00027763686)
('baggaley', 0.00027763666)
('

In [41]:

for i in range(3593):
    if l0[i][0]== "wake":
            print(i)

746


In [42]:

for i in range(3593):
    if l1[i][0]== "wake":
            print(i)

1180


In [43]:

for i in range(725, 767):
    print(l0[i])

('muse', 0.0002578964)
('generator', 0.0002578963)
('connecting', 0.00025789594)
('refine', 0.00025789576)
('multichannel', 0.00025789574)
('hearing', 0.00025789413)
('objective', 0.00025789402)
('colour', 0.00025789402)
('deadline', 0.00025789347)
('purchase', 0.00025789332)
('months', 0.00025789315)
('rune', 0.00025789213)
('references', 0.00025789192)
('risky', 0.0002578915)
('turns', 0.0002578905)
('alongside', 0.0002578904)
('discarding', 0.00025789032)
('edit', 0.00025788957)
('stein', 0.000257889)
('initial', 0.00025788872)
('ears', 0.00025788826)
('wake', 0.00025788808)
('helps', 0.00025788785)
('walk', 0.00025788764)
('configurations', 0.00025788756)
('reader', 0.00025788744)
('concerted', 0.0002578867)
('enjoy', 0.00025788636)
('desire', 0.000257886)
('wear', 0.00025788558)
('carried', 0.00025788558)
('extra', 0.00025788537)
('tedious', 0.0002578846)
('shift', 0.0002578843)
('plenty', 0.00025788406)
('alterego', 0.000257884)
('food', 0.00025788366)
('translator', 0.0002578833

In [44]:

for i in range(1159, 1201):
    print(l1[i])

('device', 0.00027798626)
('enabling', 0.00027798407)
('adolescent', 0.00027798026)
('inbetween', 0.00027797805)
('factsheet', 0.0002779779)
('translator', 0.00027797758)
('food', 0.00027797607)
('plenty', 0.00027797418)
('alterego', 0.0002779741)
('shift', 0.00027797258)
('tedious', 0.0002779708)
('extra', 0.00027796693)
('carried', 0.0002779661)
('wear', 0.00027796603)
('desire', 0.00027796428)
('enjoy', 0.00027796158)
('concerted', 0.0002779601)
('reader', 0.00027795605)
('configurations', 0.0002779556)
('walk', 0.00027795497)
('helps', 0.00027795392)
('wake', 0.00027795267)
('ears', 0.00027795162)
('initial', 0.00027794924)
('stein', 0.00027794775)
('edit', 0.00027794484)
('discarding', 0.00027794077)
('alongside', 0.00027794024)
('turns', 0.00027793995)
('risky', 0.0002779344)
('references', 0.00027793212)
('rune', 0.00027793096)
('months', 0.00027792592)
('purchase', 0.00027792476)
('deadline', 0.00027792394)
('colour', 0.000277921)
('objective', 0.00027792098)
('hearing', 0.0002

In [51]:

for i in range(3593):
    if l0[i][0]== "fivebeat":
            print(i)

2316


In [52]:

for i in range(3593):
    if l1[i][0]== "fivebeat":
            print(i)

2521


In [53]:

for i in range(2295, 2337):
    print(l0[i])

('bloc', 0.00015468955)
('sophia', 0.00015468948)
('london—seems', 0.00015468946)
('așa', 0.00015468943)
('looms', 0.00015468942)
('reciter', 0.00015468938)
('resituates', 0.00015468938)
('lacks', 0.00015468938)
('reflected', 0.00015468935)
('noon', 0.00015468935)
('obfuscation', 0.00015468927)
('endtime', 0.00015468927)
('regional', 0.00015468927)
('goal—', 0.00015468926)
('polarity', 0.00015468925)
('pana', 0.00015468923)
('housed', 0.0001546892)
('archaeology', 0.0001546892)
('woodside', 0.00015468919)
('contracted', 0.00015468914)
('chipping', 0.00015468907)
('fivebeat', 0.00015468906)
('errors', 0.00015468903)
('selective', 0.00015468903)
('drum', 0.00015468901)
('appeared', 0.00015468901)
('catchsome', 0.00015468901)
('laps', 0.000154689)
('interjects', 0.00015468898)
('fields', 0.00015468897)
('clock', 0.00015468895)
('website', 0.00015468894)
('hufnagel', 0.00015468891)
('hates', 0.00015468891)
('godhungry', 0.0001546889)
('preverbs', 0.00015468888)
('translate', 0.00015468887)

In [54]:

for i in range(2500, 2542):
    print(l1[i])

('trece', 0.00027746713)
('ageless', 0.0002774671)
('freedom', 0.00027746704)
('microcommunity', 0.00027746693)
('garden—dryden', 0.0002774669)
('translate', 0.00027746684)
('resonates', 0.0002774668)
('preverbs', 0.00027746666)
('hufnagel', 0.00027746666)
('hates', 0.00027746646)
('godhungry', 0.00027746643)
('fields', 0.0002774663)
('website', 0.00027746623)
('clock', 0.0002774662)
('laps', 0.00027746617)
('interjects', 0.00027746614)
('drum', 0.0002774661)
('catchsome', 0.00027746608)
('errors', 0.000277466)
('appeared', 0.00027746594)
('selective', 0.00027746588)
('fivebeat', 0.00027746582)
('chipping', 0.0002774656)
('contracted', 0.0002774653)
('housed', 0.00027746498)
('archaeology', 0.00027746495)
('pana', 0.00027746492)
('woodside', 0.00027746486)
('goal—', 0.00027746483)
('polarity', 0.00027746474)
('endtime', 0.0002774646)
('obfuscation', 0.00027746457)
('regional', 0.0002774645)
('noon', 0.00027746422)
('reflected', 0.00027746413)
('lacks', 0.00027746407)
('resituates', 0.0

In [56]:

for i in range(3593):
    if l0[i][0]== "blood—":
            print(i)

2924


In [57]:

for i in range(3593):
    if l1[i][0]== "blood—":
            print(i)

1911


In [58]:

for i in range(2903, 2945):
    print(l0[i])

('patient', 0.00015467731)
('microclimates', 0.0001546773)
('indicate', 0.00015467727)
('frustrations', 0.00015467725)
('reason', 0.00015467724)
('incontinence', 0.00015467721)
('drive', 0.00015467721)
('sell', 0.00015467717)
('publichtml', 0.00015467714)
('andor', 0.00015467711)
('outside', 0.0001546771)
('eugenio', 0.00015467705)
('halves', 0.000154677)
('realize', 0.000154677)
('names', 0.00015467699)
('crowded', 0.00015467693)
('welcomed', 0.00015467693)
('index', 0.00015467693)
('divergent', 0.00015467692)
('article', 0.00015467685)
('offered', 0.00015467685)
('blood—', 0.00015467679)
('bawdy', 0.00015467679)
('dislike', 0.00015467677)
('beware', 0.00015467676)
('horror', 0.00015467675)
('început', 0.00015467675)
('sides', 0.00015467669)
('tomis—ovid', 0.00015467669)
('uniquely', 0.00015467667)
('methodologies', 0.00015467664)
('recite', 0.00015467664)
('runway', 0.00015467663)
('resonance', 0.00015467661)
('chris', 0.00015467656)
('crisis', 0.00015467647)
('carearanjează', 0.0001

In [59]:

for i in range(1890, 1932):
    print(l1[i])

('have—', 0.00027753334)
('consequence', 0.00027753328)
('wordofmouth', 0.00027753325)
('mindful', 0.00027753317)
('cause', 0.00027753314)
('contamination', 0.00027753276)
('periodical', 0.00027753267)
('ejournal', 0.00027753264)
('crisis', 0.00027753258)
('carearanjează', 0.00027753238)
('chris', 0.00027753218)
('resonance', 0.00027753183)
('methodologies', 0.0002775318)
('recite', 0.00027753177)
('runway', 0.0002775316)
('sides', 0.00027753157)
('tomis—ovid', 0.0002775315)
('uniquely', 0.0002775314)
('început', 0.0002775312)
('horror', 0.00027753116)
('dislike', 0.00027753104)
('blood—', 0.000277531)
('beware', 0.00027753095)
('bawdy', 0.0002775309)
('offered', 0.0002775306)
('article', 0.00027753046)
('index', 0.00027753023)
('divergent', 0.00027753014)
('welcomed', 0.00027753008)
('crowded', 0.00027753008)
('names', 0.00027752982)
('halves', 0.00027752973)
('realize', 0.00027752965)
('eugenio', 0.0002775294)
('outside', 0.00027752935)
('andor', 0.00027752924)
('publichtml', 0.00027

In [60]:

for i in range(3593):
    if l0[i][0]== "drumsticks":
            print(i)

2800


In [61]:

for i in range(3593):
    if l1[i][0]== "drumsticks":
            print(i)

2036


In [62]:

for i in range(2779, 2821):
    print(l0[i])

('hybridized', 0.00015468002)
('what—a', 0.00015467998)
('advantageous', 0.00015467998)
('ubiquitous', 0.00015467995)
('compositions', 0.00015467989)
('telephone', 0.00015467989)
('equation', 0.00015467989)
('nations', 0.00015467987)
('example—maybe', 0.00015467986)
('nature—', 0.00015467984)
('regarding', 0.00015467984)
('renewal', 0.00015467984)
('represent', 0.00015467984)
('tradition', 0.00015467982)
('torpor', 0.00015467982)
('utter', 0.0001546798)
('rings', 0.00015467977)
('bucurii', 0.00015467974)
('routes', 0.00015467974)
('lofi', 0.00015467973)
('pandemic', 0.0001546797)
('drumsticks', 0.00015467963)
('publishers', 0.00015467963)
('eclipsing', 0.00015467954)
('styles', 0.00015467945)
('bowl', 0.00015467945)
('limitation', 0.00015467942)
('convertible', 0.00015467942)
('hudson', 0.00015467941)
('block', 0.00015467941)
('circuite', 0.00015467938)
('cannibalism', 0.00015467938)
('ticking', 0.00015467935)
('infinitely', 0.00015467935)
('dreaming', 0.0001546793)
('found—and', 0.000

In [63]:

for i in range(2015, 2057):
    print(l1[i])

('clues', 0.0002775183)
('unstuck', 0.00027751827)
('zimbri', 0.0002775182)
('duce', 0.00027751818)
('fanzine', 0.00027751797)
('ambition', 0.00027751777)
('found—and', 0.0002775177)
('sobers', 0.00027751768)
('member', 0.00027751763)
('infinitely', 0.00027751754)
('ticking', 0.00027751748)
('dreaming', 0.00027751748)
('cannibalism', 0.00027751713)
('circuite', 0.00027751707)
('block', 0.00027751704)
('convertible', 0.000277517)
('hudson', 0.00027751693)
('limitation', 0.00027751687)
('styles', 0.00027751687)
('bowl', 0.00027751678)
('eclipsing', 0.00027751608)
('drumsticks', 0.00027751588)
('publishers', 0.00027751582)
('pandemic', 0.00027751556)
('lofi', 0.0002775155)
('bucurii', 0.00027751512)
('rings', 0.000277515)
('routes', 0.000277515)
('utter', 0.00027751495)
('tradition', 0.00027751483)
('represent', 0.0002775148)
('torpor', 0.00027751474)
('example—maybe', 0.00027751471)
('nature—', 0.00027751466)
('regarding', 0.00027751463)
('renewal', 0.00027751463)
('telephone', 0.0002775

In [67]:

for i in range(3593):
    if l0[i][0]== "transport":
            print(i)

1297


In [68]:
for i in range(3593):
    if l1[i][0]== "transport":
            print(i)

3540


In [69]:

for i in range(1276, 1318):
    print(l0[i])

('fluctuation', 0.000154724)
('nine', 0.00015472372)
('occasionally', 0.00015472369)
('science', 0.00015472344)
('risk', 0.00015472338)
('wrpi', 0.00015472328)
('supplement', 0.0001547231)
('stream', 0.00015472305)
('occurring', 0.00015472302)
('laocoön', 0.00015472287)
('streamlined', 0.00015472287)
('catalog', 0.00015472285)
('risks', 0.0001547228)
('—protagonist', 0.0001547227)
('coinciding', 0.00015472261)
('chapbooks', 0.0001547226)
('timeconsuming', 0.00015472254)
('revolutioninterregnum', 0.00015472223)
('dancers', 0.00015472216)
('represented', 0.00015472202)
('desktop', 0.00015472202)
('transport', 0.00015472193)
('shore', 0.00015472175)
('etymology', 0.00015472165)
('attending', 0.00015472165)
('silly', 0.00015472165)
('judgment', 0.00015472154)
('disc', 0.00015472139)
('sharp', 0.00015472125)
('titlul', 0.0001547212)
('determined', 0.00015472114)
('aerului', 0.00015472082)
('omission', 0.00015472074)
('moving', 0.00015472072)
('envisage', 0.00015472056)
('selfindulgence', 0.

In [70]:

for i in range(3519, 3561):
    print(l1[i])

('players', 0.0002773051)
('pună', 0.000277305)
('checking', 0.000277305)
('hunting', 0.00027730284)
('photography', 0.00027730156)
('requires', 0.00027729958)
('ajunse', 0.00027729952)
('envisage', 0.00027729833)
('selfindulgence', 0.00027729824)
('moving', 0.00027729757)
('omission', 0.00027729754)
('aerului', 0.00027729696)
('determined', 0.00027729513)
('titlul', 0.00027729495)
('sharp', 0.00027729463)
('disc', 0.00027729408)
('judgment', 0.00027729315)
('attending', 0.00027729254)
('etymology', 0.00027729254)
('silly', 0.00027729242)
('shore', 0.00027729178)
('transport', 0.00027729105)
('desktop', 0.00027729064)
('represented', 0.00027729062)
('dancers', 0.0002772898)
('revolutioninterregnum', 0.00027728963)
('timeconsuming', 0.00027728794)
('coinciding', 0.0002772876)
('chapbooks', 0.00027728756)
('—protagonist', 0.00027728698)
('risks', 0.00027728657)
('catalog', 0.00027728637)
('streamlined', 0.00027728613)
('laocoön', 0.00027728584)
('occurring', 0.00027728538)
('stream', 0.0

In [71]:

for i in range(3593):
    if l0[i][0]== "ignoring":
            print(i)

1733


In [72]:
for i in range(3593):
    if l1[i][0]== "ignoring":
            print(i)

3105


In [73]:

for i in range(1712, 1754):
    print(l0[i])

('taras', 0.00015470269)
('1986', 0.00015470269)
('facilitates', 0.00015470263)
('shifting', 0.00015470262)
('changing', 0.00015470255)
('lacking', 0.00015470255)
('realizării', 0.00015470253)
('documentation', 0.00015470244)
('virus', 0.00015470244)
('opinion—', 0.00015470243)
('side', 0.0001547024)
('border', 0.00015470239)
('synthesizing', 0.00015470237)
('laptop', 0.00015470236)
('rearranged', 0.00015470224)
('fellow', 0.00015470221)
('thee', 0.00015470211)
('călărie', 0.00015470205)
('sauce', 0.00015470205)
('lives—', 0.00015470201)
('xliii', 0.000154702)
('ignoring', 0.000154702)
('fluidity', 0.000154702)
('lord', 0.00015470198)
('thirty', 0.0001547019)
('drank', 0.00015470189)
('definitely', 0.00015470186)
('dies', 0.00015470186)
('linking', 0.00015470186)
('continues', 0.00015470183)
('ripped', 0.00015470182)
('dialogues', 0.00015470179)
('saturated', 0.00015470176)
('rebirth—', 0.00015470176)
('reflecting', 0.00015470167)
('theories', 0.00015470161)
('incorporates', 0.00015470

In [74]:

for i in range(3084, 3126):
    print(l1[i])

('area', 0.0002774006)
('montreal', 0.00027739923)
('activating', 0.00027739923)
('ziua', 0.0002773992)
('easier', 0.00027739917)
('incorporates', 0.00027739911)
('theories', 0.0002773989)
('reflecting', 0.00027739877)
('saturated', 0.00027739836)
('dialogues', 0.00027739813)
('ripped', 0.00027739804)
('rebirth—', 0.00027739804)
('continues', 0.0002773979)
('definitely', 0.00027739775)
('linking', 0.00027739772)
('thirty', 0.00027739763)
('dies', 0.00027739763)
('drank', 0.00027739746)
('lord', 0.0002773972)
('xliii', 0.0002773971)
('fluidity', 0.00027739705)
('ignoring', 0.00027739705)
('lives—', 0.00027739702)
('călărie', 0.00027739676)
('sauce', 0.00027739673)
('thee', 0.00027739644)
('fellow', 0.000277396)
('rearranged', 0.00027739577)
('laptop', 0.00027739507)
('synthesizing', 0.0002773948)
('side', 0.00027739475)
('border', 0.00027739472)
('opinion—', 0.00027739463)
('documentation', 0.00027739457)
('virus', 0.00027739457)
('changing', 0.00027739423)
('realizării', 0.00027739417)

In [77]:

for i in range(3593):
    if l0[i][0]== "car":
            print(i)

In [78]:

for i in range(3593):
    if l1[i][0]== "car":
            print(i)

In [79]:

for i in range(3542, 3593):
    print(l0[i])

('thing—the', 0.00015464767)
('turn', 0.00015464755)
('ovidexile', 0.00015464755)
('existence', 0.00015464715)
('toolkit', 0.00015464709)
('espionage', 0.00015464704)
('pasta', 0.00015464655)
('many—even', 0.00015464587)
('seeing', 0.00015464587)
('videos', 0.00015464574)
('effective', 0.0001546457)
('classroom—and', 0.00015464549)
('hard—', 0.00015464546)
('hologramă', 0.00015464527)
('bridled', 0.0001546452)
('party', 0.00015464507)
('prin', 0.00015464507)
('relevant', 0.00015464479)
('stifling', 0.00015464403)
('gellu', 0.00015464363)
('tobacco', 0.00015464332)
('shorter', 0.00015464323)
('dozens', 0.00015464307)
('are—', 0.00015464217)
('forgotten', 0.00015464182)
('study—', 0.00015464147)
('crash', 0.00015464134)
('valued', 0.00015464121)
('kitchen', 0.00015463962)
('damon', 0.00015463946)
('luton', 0.00015463933)
('aview', 0.00015463876)
('complet', 0.00015463844)
('partnering', 0.00015463731)
('illustrate', 0.00015463687)
('coleading', 0.00015463668)
('band', 0.0001546365)
('hum

In [80]:

for i in range(3542, 3593):
    print(l1[i])

('represented', 0.00027729062)
('dancers', 0.0002772898)
('revolutioninterregnum', 0.00027728963)
('timeconsuming', 0.00027728794)
('coinciding', 0.0002772876)
('chapbooks', 0.00027728756)
('—protagonist', 0.00027728698)
('risks', 0.00027728657)
('catalog', 0.00027728637)
('streamlined', 0.00027728613)
('laocoön', 0.00027728584)
('occurring', 0.00027728538)
('stream', 0.00027728523)
('supplement', 0.00027728488)
('wrpi', 0.00027728363)
('risk', 0.0002772834)
('science', 0.0002772831)
('nine', 0.00027728168)
('occasionally', 0.0002772816)
('fluctuation', 0.00027728025)
('simfonie', 0.00027727964)
('embark', 0.00027727854)
('broader', 0.0002772781)
('mechanism—with', 0.00027727694)
('heyday—ruins', 0.00027727598)
('migrate', 0.00027727452)
('integration', 0.00027727275)
('counterculture', 0.00027727216)
('engagements', 0.0002772694)
('asimptotic', 0.0002772687)
('appreciated', 0.00027726704)
('rocks', 0.0002772647)
('caution', 0.00027726428)
('debate—between', 0.000277263)
('favourite', 

In [21]:

for i in range(3593):
    if l0[i][0]== "regression":
            print(i)

3251


In [22]:

for i in range(3593):
    if l1[i][0]== "regression":
            print(i)

1586


In [23]:

for i in range(3230, 3272):
    print(l0[i])

('elegance', 0.00015466851)
('acquired', 0.00015466847)
('flagellate', 0.00015466845)
('bore', 0.00015466845)
('emerge', 0.00015466839)
('breathe', 0.00015466836)
('layers', 0.00015466836)
('2014', 0.00015466836)
('sustained', 0.00015466832)
('character—', 0.0001546683)
('sketchpad', 0.00015466828)
('viewing', 0.0001546682)
('keeping', 0.00015466817)
('rods', 0.00015466807)
('fear', 0.00015466806)
('wine—he', 0.000154668)
('linebased', 0.00015466796)
('challenged', 0.0001546679)
('inventive', 0.0001546679)
('reaching', 0.0001546678)
('usefully', 0.00015466777)
('regression', 0.00015466777)
('designing', 0.00015466775)
('travel', 0.00015466774)
('cleft', 0.0001546677)
('appreciation', 0.00015466766)
('appears', 0.00015466759)
('selection', 0.00015466759)
('contented', 0.00015466758)
('likely', 0.00015466756)
('andrews', 0.00015466755)
('solos—yes', 0.0001546674)
('bottles—drinks', 0.0001546674)
('crude', 0.00015466739)
('santana', 0.00015466737)
('kills', 0.00015466736)
('displayed', 0.

In [24]:

for i in range(1565, 1607):
    print(l1[i])

('blue', 0.000277582)
('retrospective', 0.000277582)
('sections', 0.0002775818)
('movement', 0.00027758154)
('rudimentary', 0.00027758142)
('xxxiii', 0.000277581)
('kills', 0.00027758084)
('bottles—drinks', 0.0002775808)
('crude', 0.0002775808)
('santana', 0.0002775808)
('displayed', 0.0002775807)
('solos—yes', 0.00027758066)
('likely', 0.00027757988)
('andrews', 0.0002775798)
('selection', 0.00027757976)
('appears', 0.00027757973)
('contented', 0.00027757973)
('cleft', 0.00027757918)
('appreciation', 0.00027757906)
('usefully', 0.00027757886)
('designing', 0.00027757883)
('regression', 0.00027757883)
('reaching', 0.0002775788)
('travel', 0.00027757874)
('inventive', 0.0002775781)
('challenged', 0.00027757804)
('linebased', 0.00027757796)
('wine—he', 0.00027757746)
('fear', 0.00027757735)
('rods', 0.00027757708)
('keeping', 0.00027757662)
('viewing', 0.00027757636)
('sketchpad', 0.00027757612)
('sustained', 0.000277576)
('character—', 0.000277576)
('layers', 0.00027757563)
('breathe', 

In [25]:

for i in range(3593):
    if l0[i][0]== "tunes":
            print(i)

1175


In [26]:

for i in range(3593):
    if l1[i][0]== "tunes":
            print(i)

731


In [27]:

for i in range(1154, 1196):
    print(l0[i])

('pursuits', 0.00025781128)
('runes', 0.00025781122)
('teasing', 0.000257811)
('xxviii', 0.0002578109)
('drama', 0.00025781058)
('historians', 0.00025781032)
('genre', 0.00025781023)
('embracing', 0.00025781008)
('epic', 0.00025780965)
('please', 0.00025780965)
('playing', 0.0002578096)
('riffs', 0.00025780883)
('active', 0.0002578087)
('organizing', 0.00025780854)
('sseum', 0.0002578083)
('coding', 0.0002578078)
('experiments', 0.0002578078)
('real', 0.00025780767)
('england', 0.00025780738)
('fred', 0.00025780723)
('places', 0.0002578071)
('tunes', 0.00025780618)
('association', 0.00025780612)
('recontextualized', 0.00025780502)
('local', 0.00025780438)
('wakes', 0.0002578039)
('moten', 0.00025780342)
('călărind', 0.0002578033)
('dawning', 0.00025780246)
('equipment', 0.00025780243)
('positions', 0.00025780164)
('fundamental', 0.00025780013)
('creativity', 0.0002577987)
('html', 0.00025779867)
('potent', 0.00025779818)
('manuscript', 0.00025779702)
('composing', 0.00025779693)
('trou

In [28]:

for i in range(710, 752):
    print(l1[i])

('jazz', 0.00027844135)
('troughs', 0.0002784388)
('composing', 0.00027843856)
('manuscript', 0.000278438)
('rhythms', 0.0002784329)
('potent', 0.00027843192)
('html', 0.00027842948)
('creativity', 0.0002784289)
('1992', 0.0002784245)
('fundamental', 0.0002784212)
('breaking', 0.00027841897)
('positions', 0.0002784134)
('equipment', 0.00027840937)
('dawning', 0.00027840925)
('disguise', 0.00027840567)
('călărind', 0.00027840477)
('moten', 0.00027840398)
('wakes', 0.0002784013)
('local', 0.00027839895)
('recontextualized', 0.00027839537)
('association', 0.0002783897)
('tunes', 0.00027838923)
('places', 0.00027838442)
('covered', 0.000278384)
('fred', 0.00027838376)
('england', 0.00027838303)
('real', 0.0002783812)
('coding', 0.0002783807)
('experiments', 0.0002783804)
('sseum', 0.00027837846)
('organizing', 0.00027837674)
('active', 0.00027837613)
('riffs', 0.00027837514)
('cassettes', 0.00027837287)
('playing', 0.0002783712)
('please', 0.00027837115)
('epic', 0.00027837083)
('embracing

In [29]:

for i in range(3593):
    if l0[i][0]== "randomly":
            print(i)

2123


In [30]:

for i in range(3593):
    if l1[i][0]== "randomly":
            print(i)

2714


In [31]:

for i in range(2102, 2144):
    print(l0[i])

('connections', 0.00015469325)
('string', 0.00015469322)
('selfmotivated', 0.00015469317)
('cultivating', 0.00015469315)
('dara', 0.00015469313)
('possible', 0.00015469313)
('metropolitan', 0.00015469313)
('wtju', 0.00015469312)
('casei', 0.00015469312)
('wealthgod', 0.00015469312)
('daily', 0.0001546931)
('shouts', 0.00015469307)
('saucy', 0.00015469306)
('door', 0.00015469303)
('died', 0.00015469303)
('monologues', 0.000154693)
('restoration', 0.000154693)
('miar', 0.00015469297)
('mail', 0.00015469294)
('timpul', 0.00015469293)
('invited', 0.0001546929)
('randomly', 0.00015469287)
('envisaged', 0.00015469285)
('share', 0.00015469283)
('sets', 0.00015469278)
('actor', 0.00015469278)
('guitars', 0.00015469277)
('embedded', 0.00015469272)
('midwifery', 0.00015469272)
('phoneme', 0.00015469271)
('kostelanetz', 0.00015469271)
('lenses', 0.00015469271)
('illumined', 0.0001546927)
('exciting', 0.00015469265)
('warlings', 0.00015469262)
('joint', 0.0001546926)
('explorable', 0.0001546926)
(

In [32]:

for i in range(2693, 2735):
    print(l1[i])

('tragedies', 0.00027744737)
('duplicate', 0.00027744734)
('jacket2', 0.00027744734)
('needs', 0.0002774473)
('traps', 0.00027744714)
('grindini', 0.00027744705)
('explorable', 0.00027744693)
('joint', 0.00027744687)
('warlings', 0.00027744682)
('exciting', 0.00027744664)
('illumined', 0.00027744644)
('kostelanetz', 0.00027744644)
('phoneme', 0.0002774463)
('lenses', 0.00027744626)
('embedded', 0.00027744618)
('midwifery', 0.0002774461)
('actor', 0.00027744606)
('guitars', 0.00027744594)
('share', 0.0002774459)
('sets', 0.0002774459)
('envisaged', 0.0002774457)
('randomly', 0.00027744542)
('invited', 0.00027744542)
('timpul', 0.00027744513)
('mail', 0.00027744504)
('monologues', 0.00027744498)
('miar', 0.00027744492)
('restoration', 0.0002774448)
('died', 0.00027744478)
('door', 0.0002774447)
('saucy', 0.00027744446)
('shouts', 0.00027744446)
('daily', 0.00027744434)
('dara', 0.0002774442)
('possible', 0.00027744417)
('casei', 0.00027744417)
('wealthgod', 0.00027744417)
('cultivating',

In [36]:

for i in range(3593):
    if l0[i][0]== "speech":
            print(i)

911


In [46]:

for i in range(3593):
    if l0[i][0]== "commonspeech":
            print(i)

3298


In [47]:
for i in range(3593):
    if l1[i][0]== "commonspeech":
            print(i)

1539


In [48]:

for i in range(3277, 3319):
    print(l0[i])

('virus', 0.00027749196)
('emerging', 0.00027749196)
('loan', 0.00027749193)
('vesuvius', 0.00027749187)
('conspired', 0.0002774917)
('grow', 0.0002774917)
('thee', 0.0002774914)
('frank', 0.00027749137)
('elvis', 0.00027749132)
('altcuiva', 0.00027749126)
('away—he', 0.0002774907)
('heralding', 0.0002774906)
('rapid', 0.00027749047)
('pună', 0.0002774904)
('blank', 0.00027749036)
('contain', 0.00027749012)
('anybody', 0.00027749)
('epsom', 0.00027749)
('tatters', 0.00027749)
('revolutioninterregnum', 0.0002774899)
('publichtml', 0.0002774899)
('commonspeech', 0.00027748971)
('track', 0.00027748963)
('embedded', 0.00027748937)
('amendments', 0.00027748887)
('view', 0.00027748826)
('surprise', 0.00027748817)
('nuanced', 0.00027748806)
('policers', 0.00027748782)
('recorder—doing', 0.00027748753)
('onthefly', 0.00027748747)
('fourteen', 0.00027748742)
('cent', 0.00027748724)
('successful', 0.00027748698)
('219', 0.00027748683)
('gellu', 0.0002774868)
('lots', 0.00027748678)
('capable', 0

In [49]:

for i in range(1518, 1560):
    print(l1[i])

('shooting', 0.00015470809)
('ești', 0.00015470796)
('except', 0.00015470793)
('poststructuralist', 0.00015470793)
('statement', 0.00015470787)
('gellu', 0.00015470784)
('capable', 0.00015470784)
('lots', 0.00015470784)
('219', 0.00015470783)
('successful', 0.0001547078)
('cent', 0.00015470776)
('fourteen', 0.00015470773)
('recorder—doing', 0.00015470768)
('onthefly', 0.00015470768)
('policers', 0.00015470762)
('nuanced', 0.00015470761)
('surprise', 0.00015470758)
('view', 0.00015470752)
('amendments', 0.00015470738)
('track', 0.00015470733)
('embedded', 0.00015470732)
('commonspeech', 0.00015470729)
('revolutioninterregnum', 0.00015470726)
('epsom', 0.00015470723)
('publichtml', 0.00015470723)
('tatters', 0.0001547072)
('contain', 0.0001547072)
('anybody', 0.00015470719)
('blank', 0.00015470716)
('rapid', 0.00015470716)
('pună', 0.00015470713)
('heralding', 0.00015470712)
('away—he', 0.00015470709)
('thee', 0.00015470697)
('frank', 0.00015470697)
('altcuiva', 0.00015470697)
('elvis', 

In [50]:

len(l0)

3594

In [51]:

for i in range(3594):
    if l0[i][0]== "arcane":
            print(i)

1695


In [52]:
for i in range(3593):
    if l1[i][0]== "arcane":
            print(i)

3143


In [53]:

for i in range(1664, 1717):
    print(l0[i])

('goodbye', 0.0002776449)
('doctoral', 0.00027764463)
('primary', 0.0002776445)
('feel', 0.00027764446)
('eliminate', 0.00027764443)
('digitizing', 0.00027764437)
('seemingly', 0.00027764423)
('prose', 0.00027764373)
('compositions', 0.00027764364)
('tattered', 0.0002776436)
('pardonable', 0.0002776434)
('develop', 0.0002776432)
('resonance', 0.00027764315)
('iarba', 0.00027764295)
('dialogos', 0.00027764292)
('knife', 0.0002776428)
('centers', 0.00027764266)
('canon', 0.0002776425)
('climates', 0.00027764228)
('fits', 0.0002776422)
('forthcoming', 0.0002776421)
('countries', 0.0002776421)
('earnest', 0.00027764193)
('occasionally', 0.00027764187)
('succinctly', 0.00027764184)
('wellknown', 0.0002776417)
('foto', 0.00027764135)
('frieze', 0.0002776412)
('rubbing', 0.00027764088)
('underway', 0.00027764088)
('service', 0.00027764068)
('arcane', 0.00027764068)
('switch', 0.0002776405)
('sauce', 0.0002776405)
('involving', 0.00027764044)
('irreverence', 0.00027764036)
('angoasei', 0.00027

In [54]:

for i in range(3122, 3164):
    print(l1[i])

('token', 0.00015467918)
('editorship', 0.00015467916)
('perceived', 0.00015467915)
('deapoi', 0.00015467915)
('current', 0.0001546791)
('concerned', 0.00015467909)
('riposte', 0.00015467909)
('famous', 0.00015467906)
('responsibility—to', 0.00015467906)
('admit', 0.00015467904)
('vivid', 0.00015467903)
('waldman', 0.00015467903)
('displayed', 0.00015467902)
('two—and', 0.000154679)
('angoasei', 0.00015467894)
('irreverence', 0.00015467894)
('inclined', 0.00015467894)
('involving', 0.00015467891)
('switch', 0.0001546789)
('service', 0.0001546789)
('sauce', 0.0001546789)
('arcane', 0.00015467888)
('underway', 0.00015467884)
('rubbing', 0.00015467881)
('frieze', 0.0001546788)
('foto', 0.00015467872)
('wellknown', 0.00015467867)
('succinctly', 0.00015467865)
('occasionally', 0.00015467865)
('earnest', 0.00015467862)
('countries', 0.00015467861)
('forthcoming', 0.00015467861)
('fits', 0.00015467854)
('climates', 0.00015467852)
('centers', 0.0001546785)
('canon', 0.0001546785)
('knife', 0.0

In [56]:

for i in range(3594):
    if l0[i][0]== "constrict":
            print(i)

1980


In [57]:

for i in range(3594):
    if l1[i][0]== "constrict":
            print(i)

2856


In [29]:

for i in range(3581):
    if l0[i][0]== "predetermined":
            print(i)

3028


In [32]:

for i in range(3581):
    if l0[i][0]== "skill":
            print(i)

744


In [33]:

for i in range(3581):
    if l1[i][0]== "skill":
            print(i)

1185


In [34]:

for i in range(713, 765):
              print(l0[i])

('playing', 0.00025842636)
('british', 0.00025842467)
('previously', 0.00025842403)
('objective', 0.0002584236)
('connect', 0.00025842144)
('references', 0.00025842144)
('empty', 0.00025841905)
('transcription', 0.00025841882)
('acts', 0.00025841792)
('composition', 0.00025841792)
('peoples', 0.00025841736)
('html', 0.0002584172)
('taner', 0.00025841675)
('tool', 0.00025841527)
('javascript', 0.00025841463)
('academia', 0.0002584143)
('multidecade', 0.00025841428)
('combined', 0.00025841393)
('understanding', 0.00025841364)
('similar', 0.0002584128)
('1987', 0.00025841268)
('investing', 0.000258412)
('color', 0.0002584118)
('factsheet', 0.00025841052)
('recontextualized', 0.00025841026)
('community', 0.00025841026)
('contained', 0.00025841015)
('guess', 0.00025841003)
('deep', 0.0002584099)
('tales', 0.00025840907)
('operations', 0.00025840878)
('skill', 0.0002584074)
('feeling', 0.0002584073)
('stuck', 0.00025840677)
('src', 0.00025840665)
('zero', 0.00025840648)
('samples', 0.0002584

In [35]:

for i in range(1164, 1206):
              print(l1[i])

('bored', 0.00027879715)
('lack', 0.00027879662)
('turned', 0.00027879523)
('piss', 0.0002787946)
('history', 0.00027879397)
('human', 0.0002787934)
('guitar', 0.00027879182)
('stated', 0.0002787888)
('runes', 0.00027878647)
('alongside', 0.00027878516)
('conversation', 0.00027878452)
('roughly', 0.00027878396)
('method', 0.00027877965)
('unde', 0.00027877817)
('obscure', 0.00027877686)
('specific', 0.00027877637)
('samples', 0.000278776)
('zero', 0.0002787757)
('src', 0.00027877427)
('stuck', 0.0002787739)
('feeling', 0.000278771)
('skill', 0.00027877055)
('operations', 0.00027876304)
('tales', 0.00027876173)
('deep', 0.00027875675)
('guess', 0.00027875637)
('contained', 0.00027875594)
('recontextualized', 0.00027875532)
('community', 0.00027875495)
('factsheet', 0.0002787539)
('color', 0.00027874738)
('investing', 0.00027874572)
('1987', 0.00027874234)
('similar', 0.00027874167)
('understanding', 0.00027873728)
('combined', 0.00027873556)
('multidecade', 0.00027873393)
('academia', 0

In [36]:

for i in range(3581):
    if l0[i][0]== "sifting":
            print(i)

1668


In [37]:

for i in range(3581):
    if l1[i][0]== "sifting":
            print(i)

3160


In [38]:

for i in range(1647, 1689):
              print(l0[i])

('amazon', 0.00015502032)
('operation', 0.0001550203)
('chipping', 0.00015502027)
('crowded', 0.00015502027)
('încă', 0.00015502026)
('fanzine', 0.00015502024)
('catching', 0.0001550202)
('passes', 0.0001550201)
('generated', 0.00015502007)
('worldwide', 0.00015502005)
('mediumrelated', 0.00015502)
('opinion—mere', 0.00015501998)
('orchestrating', 0.00015501992)
('majorminor—', 0.00015501982)
('anything', 0.00015501982)
('incompetent', 0.00015501972)
('outdated', 0.0001550197)
('—protagonist', 0.00015501968)
('streamlined', 0.0001550196)
('caring', 0.0001550196)
('illuminates', 0.00015501957)
('sifting', 0.00015501957)
('existence', 0.00015501952)
('shock—', 0.0001550195)
('prose', 0.0001550195)
('discourse—even', 0.00015501949)
('behind', 0.00015501941)
('proper', 0.00015501933)
('disc', 0.00015501921)
('amendments', 0.00015501921)
('instructive', 0.00015501914)
('variation', 0.00015501911)
('proximate', 0.00015501911)
('aerului', 0.00015501911)
('riff', 0.00015501911)
('capacity', 

In [39]:

for i in range(3139, 3181):
              print(l1[i])

('sketchpad', 0.0002780058)
('righting', 0.00027800564)
('trebuie', 0.000278005)
('lacks', 0.00027800477)
('reaching', 0.00027800445)
('capacity', 0.00027800436)
('riff', 0.0002780036)
('aerului', 0.00027800357)
('proximate', 0.0002780035)
('variation', 0.00027800346)
('instructive', 0.00027800322)
('disc', 0.00027800296)
('amendments', 0.00027800282)
('proper', 0.00027800226)
('behind', 0.00027800165)
('shock—', 0.0002780015)
('discourse—even', 0.00027800148)
('existence', 0.0002780014)
('prose', 0.00027800133)
('illuminates', 0.000278001)
('streamlined', 0.00027800084)
('sifting', 0.00027800084)
('caring', 0.00027800084)
('—protagonist', 0.00027800055)
('outdated', 0.00027800037)
('incompetent', 0.00027800025)
('majorminor—', 0.00027799982)
('anything', 0.0002779997)
('orchestrating', 0.0002779991)
('mediumrelated', 0.00027799883)
('opinion—mere', 0.0002779988)
('generated', 0.00027799845)
('passes', 0.00027799825)
('worldwide', 0.00027799822)
('catching', 0.00027799763)
('crowded', 

In [40]:

for i in range(3581):
    if l0[i][0]== "curse":
            print(i)

3215


In [41]:

for i in range(3581):
    if l1[i][0]== "curse":
            print(i)

1611


In [42]:

for i in range(3194, 3236):
              print(l0[i])

('efficient', 0.00015497641)
('interval', 0.00015497638)
('overlooked', 0.00015497633)
('discouraging', 0.00015497631)
('attending', 0.00015497608)
('explorations', 0.00015497606)
('halfway', 0.00015497603)
('variously', 0.000154976)
('dawn', 0.00015497599)
('supplement', 0.00015497599)
('personally', 0.00015497599)
('dintre', 0.00015497596)
('fourteen', 0.00015497593)
('guided', 0.00015497592)
('inconsistent', 0.00015497589)
('ours', 0.00015497585)
('twice', 0.00015497585)
('experimental', 0.00015497579)
('dries', 0.00015497574)
('prescient', 0.00015497574)
('density', 0.00015497571)
('curse', 0.00015497571)
('bluster', 0.00015497569)
('exorbitant', 0.00015497567)
('persevere', 0.0001549756)
('dies', 0.00015497557)
('protagonists', 0.00015497551)
('possessing', 0.00015497544)
('renewal', 0.00015497534)
('news', 0.00015497528)
('anglosaxon', 0.00015497525)
('legroom', 0.00015497523)
('containing', 0.00015497519)
('intersecție', 0.00015497515)
('mistreated', 0.00015497507)
('zeului', 0

In [43]:

for i in range(1590, 1632):
              print(l1[i])

('elizabethanrestoration', 0.00027823955)
('onstage', 0.00027823928)
('loop', 0.0002782391)
('layering', 0.0002782391)
('sadly', 0.0002782387)
('fumbling', 0.0002782379)
('web—might', 0.00027823768)
('zeului', 0.00027823733)
('mistreated', 0.0002782371)
('intersecție', 0.00027823675)
('containing', 0.00027823646)
('anglosaxon', 0.00027823634)
('legroom', 0.00027823626)
('news', 0.00027823605)
('renewal', 0.0002782358)
('possessing', 0.00027823524)
('protagonists', 0.00027823483)
('dies', 0.00027823454)
('persevere', 0.0002782345)
('exorbitant', 0.00027823407)
('bluster', 0.0002782339)
('curse', 0.0002782338)
('density', 0.00027823378)
('prescient', 0.00027823375)
('dries', 0.00027823355)
('experimental', 0.0002782334)
('twice', 0.00027823306)
('ours', 0.0002782329)
('guided', 0.00027823282)
('inconsistent', 0.0002782327)
('fourteen', 0.00027823268)
('dintre', 0.00027823247)
('personally', 0.00027823244)
('supplement', 0.0002782324)
('dawn', 0.00027823233)
('variously', 0.00027823227)


In [26]:

for i in range(3647):
    if l0[i][0]== "shunned":
            print(i)

1714


In [27]:

for i in range(3647):
    if l1[i][0]== "shunned":
            print(i)

3151


In [28]:

for i in range(1693, 1735):
              print(l0[i])

('derbydays', 0.0002732741)
('emphasises', 0.00027327397)
('affective', 0.00027327394)
('diminish', 0.00027327376)
('warmer', 0.00027327376)
('microcollections', 0.00027327362)
('dimension', 0.0002732735)
('beddeddown', 0.00027327298)
('transcreation', 0.0002732725)
('hotel', 0.00027327178)
('nations', 0.00027327167)
('pânla', 0.0002732709)
('dreaming', 0.00027327068)
('before—', 0.0002732703)
('factsheet', 0.00027327024)
('cover', 0.00027327013)
('woke', 0.0002732698)
('tide—', 0.00027326954)
('torchgod', 0.0002732693)
('relevant', 0.0002732692)
('procedură', 0.00027326905)
('shunned', 0.00027326884)
('cunning', 0.0002732686)
('ours', 0.00027326855)
('snappy', 0.00027326847)
('african', 0.0002732684)
('relevance', 0.00027326835)
('woodside', 0.0002732683)
('lorrain', 0.0002732682)
('dread', 0.00027326812)
('conceptual', 0.0002732681)
('unaware', 0.00027326806)
('ethos', 0.000273268)
('anew', 0.00027326788)
('list', 0.00027326783)
('conjunction', 0.00027326727)
('grate', 0.000273267)

In [29]:

for i in range(3130, 3172):
              print(l1[i])

('lathy', 0.00015714167)
('unstuck', 0.00015714167)
('strangled', 0.00015714162)
('grate', 0.00015714146)
('conjunction', 0.00015714142)
('anew', 0.00015714132)
('list', 0.00015714132)
('unaware', 0.00015714129)
('conceptual', 0.00015714129)
('ethos', 0.00015714127)
('dread', 0.00015714124)
('lorrain', 0.00015714124)
('woodside', 0.00015714124)
('african', 0.00015714123)
('relevance', 0.00015714118)
('ours', 0.00015714117)
('snappy', 0.00015714117)
('cunning', 0.00015714111)
('procedură', 0.0001571411)
('relevant', 0.00015714105)
('torchgod', 0.00015714104)
('shunned', 0.00015714104)
('tide—', 0.000157141)
('woke', 0.00015714095)
('cover', 0.0001571409)
('before—', 0.00015714085)
('factsheet', 0.00015714084)
('dreaming', 0.00015714075)
('pânla', 0.0001571407)
('hotel', 0.00015714056)
('nations', 0.00015714054)
('transcreation', 0.00015714038)
('beddeddown', 0.0001571403)
('dimension', 0.00015714021)
('microcollections', 0.0001571402)
('affective', 0.00015714014)
('warmer', 0.00015714

In [30]:

for i in range(3647):
    if l0[i][0]== "abroad":
            print(i)

3381


In [31]:
for i in range(3647):
    if l1[i][0]== "abroad":
            print(i)

1483


In [32]:

for i in range(3360, 3402):
              print(l0[i])

('documentaryaudio', 0.00027304102)
('grown', 0.00027304085)
('honour', 0.00027304076)
('algorithm', 0.00027304073)
('hobby', 0.0002730406)
('are—', 0.00027304055)
('muggers—', 0.00027304035)
('existence', 0.0002730403)
('majesty', 0.0002730403)
('poeticdiscourse', 0.00027303965)
('shed', 0.00027303942)
('tenets', 0.00027303895)
('stifling', 0.0002730389)
('leaf', 0.00027303887)
('alvin', 0.00027303875)
('deserves', 0.0002730385)
('plant', 0.00027303837)
('muffle', 0.00027303834)
('words—', 0.00027303808)
('geographical', 0.00027303796)
('perceived', 0.00027303796)
('abroad', 0.0002730379)
('selfmotivated', 0.00027303776)
('ie', 0.00027303764)
('drum', 0.0002730374)
('subsaharan', 0.00027303718)
('linguistically', 0.0002730371)
('household', 0.00027303683)
('tobacco', 0.00027303645)
('institution', 0.00027303636)
('fall', 0.0002730359)
('saucy', 0.00027303584)
('wanted', 0.00027303584)
('pose', 0.00027303578)
('leaving', 0.00027303555)
('happened', 0.00027303497)
('forgotten', 0.000273

In [33]:

for i in range(1462, 1504):
              print(l1[i])

('actor', 0.00015718688)
('develop', 0.00015718683)
('discusses', 0.00015718675)
('curtain', 0.00015718672)
('forgotten', 0.00015718669)
('happened', 0.00015718656)
('leaving', 0.0001571865)
('saucy', 0.00015718644)
('wanted', 0.00015718641)
('fall', 0.00015718641)
('pose', 0.00015718641)
('institution', 0.00015718632)
('tobacco', 0.0001571863)
('household', 0.00015718624)
('linguistically', 0.0001571862)
('subsaharan', 0.00015718616)
('ie', 0.0001571861)
('drum', 0.00015718606)
('selfmotivated', 0.00015718605)
('geographical', 0.00015718602)
('perceived', 0.00015718602)
('abroad', 0.00015718602)
('words—', 0.000157186)
('plant', 0.00015718593)
('muffle', 0.00015718592)
('deserves', 0.00015718592)
('alvin', 0.00015718582)
('leaf', 0.00015718582)
('stifling', 0.00015718582)
('tenets', 0.00015718579)
('shed', 0.00015718574)
('poeticdiscourse', 0.00015718571)
('majesty', 0.00015718557)
('existence', 0.00015718557)
('muggers—', 0.00015718554)
('hobby', 0.00015718551)
('are—', 0.00015718551

In [34]:

for i in range(3647):
    if l0[i][0]== "minor":
            print(i)

201


In [35]:
for i in range(3647):
    if l1[i][0]== "minor":
            print(i)

212


In [36]:

for i in range(180, 222):
              print(l0[i])

('electronic', 0.00027829662)
('after', 0.00027827456)
('people', 0.0002782712)
('agree', 0.00027826565)
('power', 0.0002782571)
('performance', 0.0002782091)
('network', 0.0002781543)
('naropa', 0.00027814668)
('death', 0.00027812482)
('full', 0.000278111)
('access', 0.00027806373)
('applied', 0.00027805698)
('level', 0.0002780486)
('uniting', 0.0002780432)
('group', 0.00027803984)
('efforts', 0.00027803276)
('diddley', 0.0002779822)
('pattern', 0.00027795622)
('editors', 0.00027792886)
('never', 0.00027792106)
('published', 0.0002778932)
('minor', 0.00027789167)
('beyond', 0.00027786597)
('truth', 0.0002778314)
('useful', 0.00027782834)
('fine', 0.00027781984)
('documentary', 0.00027775558)
('book', 0.00027774757)
('until', 0.00027773622)
('contemporary', 0.0002777344)
('perfect', 0.00027768442)
('went', 0.000277675)
('computers', 0.00027766597)
('2012', 0.0002776626)
('towards', 0.0002776595)
('process', 0.0002776591)
('despair', 0.00027764624)
('night', 0.00027764085)
('poor', 0.00

In [41]:

for i in range(3647):
    if l0[i][0]== "canon":
            print(i)

2869


In [42]:

for i in range(3647):
    if l1[i][0]== "canon":
            print(i)

1993


In [43]:

for i in range(2848, 2890):
              print(l0[i])

('hefty', 0.00027312408)
('foremost', 0.00027312408)
('separated', 0.00027312382)
('inspired', 0.00027312376)
('writer', 0.00027312362)
('halfway', 0.0002731235)
('equally', 0.00027312347)
('considering', 0.00027312335)
('regard', 0.00027312327)
('ignored', 0.0002731232)
('depths', 0.00027312292)
('repetition—', 0.00027312274)
('punk', 0.0002731227)
('nuance', 0.0002731223)
('sell', 0.00027312187)
('wikipedia', 0.00027312178)
('herein', 0.00027312175)
('possibility', 0.00027312167)
('blasting', 0.00027312135)
('potentials', 0.00027312126)
('graphical', 0.0002731211)
('canon', 0.00027312097)
('found—and', 0.0002731207)
('njit', 0.00027312047)
('watergift', 0.0002731204)
('scattered', 0.00027312033)
('viable', 0.00027312012)
('repast', 0.00027312007)
('suprafața', 0.00027311995)
('couldn', 0.00027311995)
('computational', 0.0002731199)
('poetul', 0.0002731198)
('ripped', 0.00027311975)
('scenele', 0.0002731196)
('visualdesign', 0.0002731195)
('takeitback', 0.00027311945)
('tutun', 0.000

In [44]:

for i in range(1972, 2004):
              print(l1[i])

('awaiting', 0.00015717033)
('leather', 0.0001571703)
('bright', 0.0001571703)
('substance', 0.0001571703)
('tutun', 0.00015717022)
('receiver', 0.0001571702)
('scenele', 0.00015717017)
('halfbaked', 0.00015717017)
('takeitback', 0.00015717016)
('visualdesign', 0.00015717014)
('ripped', 0.00015717013)
('poetul', 0.0001571701)
('suprafața', 0.0001571701)
('couldn', 0.00015717007)
('computational', 0.00015717007)
('viable', 0.00015717006)
('repast', 0.00015717006)
('watergift', 0.00015717)
('scattered', 0.00015717)
('njit', 0.00015716995)
('found—and', 0.00015716992)
('canon', 0.00015716988)
('graphical', 0.00015716985)
('potentials', 0.00015716981)
('blasting', 0.00015716978)
('herein', 0.00015716972)
('wikipedia', 0.0001571697)
('sell', 0.00015716969)
('possibility', 0.00015716969)
('nuance', 0.00015716962)
('punk', 0.00015716955)
('depths', 0.00015716949)


In [45]:

for i in range(3647):
    if l0[i][0]== "sofa":
            print(i)

3356


In [46]:
for i in range(3647):
    if l1[i][0]== "sofa":
            print(i)

1505


In [47]:

for i in range(3335, 3377):
              print(l0[i])

('gust', 0.00027304594)
('reckoning', 0.00027304582)
('founded', 0.00027304553)
('looms', 0.0002730453)
('land', 0.0002730451)
('realizării', 0.00027304483)
('ămândă', 0.0002730447)
('zimbri', 0.00027304454)
('thoroughly', 0.00027304448)
('generarea', 0.0002730442)
('foundations', 0.00027304355)
('explorable', 0.00027304352)
('reflect', 0.00027304344)
('organize', 0.00027304338)
('amplification', 0.00027304285)
('fulllength', 0.0002730428)
('trăgând', 0.00027304213)
('mâncare', 0.00027304213)
('passes', 0.00027304198)
('courses', 0.00027304192)
('territory', 0.00027304178)
('sofa', 0.00027304163)
('enclose', 0.00027304157)
('conversion', 0.00027304143)
('centers', 0.00027304125)
('documentaryaudio', 0.00027304102)
('grown', 0.00027304085)
('honour', 0.00027304076)
('algorithm', 0.00027304073)
('hobby', 0.0002730406)
('are—', 0.00027304055)
('muggers—', 0.00027304035)
('existence', 0.0002730403)
('majesty', 0.0002730403)
('poeticdiscourse', 0.00027303965)
('shed', 0.00027303942)


In [48]:

for i in range(1484, 1526):
              print(l1[i])

('words—', 0.000157186)
('plant', 0.00015718593)
('muffle', 0.00015718592)
('deserves', 0.00015718592)
('alvin', 0.00015718582)
('leaf', 0.00015718582)
('stifling', 0.00015718582)
('tenets', 0.00015718579)
('shed', 0.00015718574)
('poeticdiscourse', 0.00015718571)
('majesty', 0.00015718557)
('existence', 0.00015718557)
('muggers—', 0.00015718554)
('hobby', 0.00015718551)
('are—', 0.00015718551)
('algorithm', 0.0001571855)
('grown', 0.00015718547)
('honour', 0.00015718542)
('documentaryaudio', 0.00015718542)
('centers', 0.00015718536)
('conversion', 0.00015718534)
('sofa', 0.00015718534)
('enclose', 0.00015718532)
('territory', 0.00015718525)
('trăgând', 0.00015718525)
('passes', 0.00015718525)
('mâncare', 0.00015718522)
('courses', 0.00015718519)
('amplification', 0.00015718507)
('fulllength', 0.00015718507)
('reflect', 0.00015718497)
('organize', 0.00015718494)
('explorable', 0.00015718494)
('foundations', 0.00015718491)
('generarea', 0.00015718481)
('zimbri', 0.00015718474)
('thor

In [51]:

for i in range(3647):
    if l0[i][0]== "bottle":
            print(i)

1571


In [52]:

for i in range(3647):
    if l1[i][0]== "bottle":
            print(i)

3291


In [53]:

for i in range(1550, 1592):
              print(l0[i])

('discovered', 0.0002733013)
('resolution', 0.00027330106)
('asimptotic', 0.00027330074)
('arcane', 0.00027330065)
('replete', 0.00027330057)
('moments', 0.00027330033)
('outlined', 0.00027330001)
('definitely', 0.00027329978)
('suntem', 0.00027329972)
('curse', 0.00027329935)
('receipt', 0.00027329926)
('hybridization', 0.00027329905)
('colosseum', 0.00027329903)
('tendencies', 0.00027329882)
('contains', 0.0002732987)
('guided', 0.00027329827)
('display', 0.00027329804)
('youtube', 0.00027329795)
('ramblings', 0.00027329792)
('explained', 0.00027329786)
('exciting', 0.00027329783)
('bottle', 0.0002732977)
('precise', 0.00027329757)
('protestant', 0.0002732974)
('france', 0.0002732974)
('pulldown', 0.000273297)
('dark', 0.00027329693)
('excerpt', 0.00027329684)
('explain', 0.00027329684)
('apparatus', 0.0002732968)
('importance', 0.00027329655)
('going', 0.00027329611)
('faustus', 0.00027329606)
('risking', 0.00027329565)
('collectives', 0.00027329565)
('argue', 0.0002732953)
('dodgy'

In [54]:

for i in range(3270, 3312):
              print(l0[i])

('exercițiile', 0.00027305834)
('clear', 0.0002730582)
('decide', 0.000273058)
('significance', 0.00027305793)
('unholy', 0.0002730578)
('contented', 0.0002730577)
('audioinkineticrealms', 0.00027305746)
('institute', 0.00027305746)
('portal', 0.00027305743)
('designing', 0.0002730574)
('phrasing', 0.0002730573)
('compile', 0.00027305688)
('wealth', 0.0002730568)
('burnt', 0.00027305653)
('beginners', 0.00027305644)
('ticking', 0.00027305612)
('download', 0.00027305612)
('febra', 0.0002730559)
('allembracing', 0.00027305578)
('audioproduction', 0.00027305575)
('boldfaced', 0.00027305546)
('musicians', 0.00027305546)
('ejournal', 0.00027305522)
('transfer', 0.0002730546)
('circulate', 0.00027305444)
('morbid', 0.00027305406)
('fond', 0.00027305403)
('invited', 0.00027305388)
('primary', 0.00027305353)
('token', 0.0002730534)
('videos', 0.00027305298)
('elegance', 0.00027305295)
('phrase', 0.00027305275)
('browser', 0.00027305243)
('environmental', 0.00027305243)
('plain', 0.00027305237

In [55]:

for i in range(3647):
    if l0[i][0]== "empty":
            print(i)

736


In [56]:
for i in range(3647):
    if l1[i][0]== "empty":
            print(i)

1134


In [57]:

for i in range(715, 757):
              print(l0[i])

('extra', 0.00027437767)
('invest', 0.0002743731)
('annals', 0.0002743728)
('embed', 0.0002743727)
('college', 0.00027437168)
('created', 0.00027436952)
('hologram', 0.00027436772)
('taught', 0.00027436748)
('create', 0.00027436606)
('fund', 0.00027436495)
('converter', 0.0002743649)
('unusual', 0.0002743648)
('alter', 0.00027436437)
('marks', 0.00027436306)
('composition', 0.00027436216)
('troughs', 0.00027436216)
('longer', 0.00027436131)
('worn', 0.0002743563)
('fundamental', 0.00027435488)
('left', 0.00027435325)
('minutes', 0.00027435002)
('empty', 0.00027434833)
('increasingly', 0.00027434755)
('question', 0.00027434638)
('heard', 0.00027434173)
('alternative', 0.00027433594)
('studies', 0.00027433448)
('arranged', 0.00027433413)
('guitar', 0.00027433244)
('multichannel', 0.0002743285)
('components', 0.00027432668)
('case', 0.0002743265)
('degree', 0.00027432406)
('capturing', 0.0002743225)
('shattered', 0.0002743205)
('discuss', 0.00027431932)
('names', 0.00027431763)
('managing

In [58]:

for i in range(3647):
    if l0[i][0]== "despair":
            print(i)

216


In [60]:

for i in range(3647):
    if l1[i][0]== "despair":
            print(i)

303


In [61]:

for i in range(195, 237):
              print(l0[i])

('efforts', 0.00027803276)
('diddley', 0.0002779822)
('pattern', 0.00027795622)
('editors', 0.00027792886)
('never', 0.00027792106)
('published', 0.0002778932)
('minor', 0.00027789167)
('beyond', 0.00027786597)
('truth', 0.0002778314)
('useful', 0.00027782834)
('fine', 0.00027781984)
('documentary', 0.00027775558)
('book', 0.00027774757)
('until', 0.00027773622)
('contemporary', 0.0002777344)
('perfect', 0.00027768442)
('went', 0.000277675)
('computers', 0.00027766597)
('2012', 0.0002776626)
('towards', 0.0002776595)
('process', 0.0002776591)
('despair', 0.00027764624)
('night', 0.00027764085)
('poor', 0.00027761725)
('problem', 0.00027756547)
('fourfour', 0.0002775642)
('dislocation', 0.0002775618)
('school', 0.0002775444)
('need', 0.00027754417)
('ability', 0.00027751864)
('wrote', 0.00027751678)
('collection', 0.00027748614)
('london', 0.00027747746)
('important', 0.00027746675)
('somebody', 0.00027745758)
('critical', 0.0002774423)
('presenting', 0.00027743843)
('idea', 0.000277437

In [62]:

for i in range(282, 324):
              print(l1[i])

('play', 0.00057628314)
('theme', 0.0005762702)
('addition', 0.0005762641)
('improvisedishowthevoiceisused', 0.00057626073)
('particular', 0.0005762591)
('drummer', 0.0005762566)
('users', 0.00057625433)
('poetic', 0.00057625206)
('space', 0.00057624583)
('happen', 0.0005762435)
('albany', 0.00057624007)
('idea', 0.0005762336)
('critical', 0.0005762327)
('somebody', 0.00057622953)
('london', 0.00057622575)
('collection', 0.00057622406)
('school', 0.000576213)
('need', 0.0005762129)
('dislocation', 0.00057620957)
('fourfour', 0.00057620904)
('night', 0.00057619426)
('despair', 0.00057619315)
('computers', 0.0005761894)
('went', 0.0005761876)
('published', 0.0005761454)
('group', 0.0005761172)
('power', 0.0005760751)
('conference', 0.00057600293)
('writers', 0.0005759242)
('drinks', 0.00047183107)
('whatever', 0.00047182842)
('greater', 0.0004717318)
('controls', 0.00047172685)
('collage', 0.00047171421)
('below', 0.000471711)
('subject', 0.00047170938)
('believe', 0.0004717046)
('humour

In [63]:

for i in range(3647):
    if l0[i][0]== "transgression":
            print(i)

1390


In [64]:

for i in range(3647):
    if l1[i][0]== "transgression":
            print(i)

3472


In [65]:

for i in range(1369, 1411):
              print(l0[i])

('cats', 0.00027335)
('expressiveaudio', 0.00027335)
('coding', 0.0002733494)
('undulating', 0.00027334754)
('digitizing', 0.0002733474)
('share', 0.00027334722)
('cruel', 0.0002733467)
('peterlambornwilson', 0.00027334658)
('finale', 0.0002733465)
('sorry', 0.0002733463)
('grosz', 0.0002733461)
('maintained', 0.00027334597)
('contamination', 0.00027334562)
('strike', 0.00027334518)
('culture', 0.00027334507)
('coproduce', 0.0002733445)
('involving', 0.00027334422)
('incontinence', 0.000273344)
('expressed', 0.00027334396)
('crude', 0.00027334393)
('utter', 0.00027334364)
('transgression', 0.00027334364)
('microphones', 0.00027334286)
('către', 0.00027334245)
('speculation', 0.00027334134)
('foto', 0.00027333904)
('activategenerate', 0.00027333893)
('cigarette', 0.00027333878)
('grace', 0.00027333875)
('2000', 0.0002733387)
('alabama', 0.00027333846)
('audience', 0.00027333808)
('likeminded', 0.00027333773)
('partea', 0.00027333768)
('organizational', 0.00027333765)
('725000', 0.00027

In [66]:

for i in range(3451, 3493):
              print(l1[i])

('instrumentation', 0.00015712806)
('iluminare', 0.000157128)
('ship', 0.00015712793)
('temporarily', 0.00015712788)
('defined', 0.00015712783)
('pliant', 0.0001571278)
('begins', 0.00015712775)
('likeminded', 0.00015712772)
('organizational', 0.00015712772)
('725000', 0.00015712772)
('partea', 0.0001571277)
('audience', 0.00015712761)
('alabama', 0.00015712758)
('2000', 0.00015712752)
('cigarette', 0.0001571275)
('grace', 0.0001571275)
('activategenerate', 0.00015712748)
('foto', 0.00015712748)
('speculation', 0.000157127)
('către', 0.00015712678)
('microphones', 0.00015712672)
('transgression', 0.00015712657)
('utter', 0.00015712655)
('expressed', 0.00015712652)
('crude', 0.0001571265)
('incontinence', 0.00015712647)
('coproduce', 0.0001571264)
('involving', 0.0001571264)
('culture', 0.00015712627)
('strike', 0.00015712625)
('contamination', 0.0001571262)
('maintained', 0.00015712612)
('sorry', 0.00015712605)
('grosz', 0.00015712605)
('finale', 0.00015712602)
('peterlambornwilson', 

In [67]:

for i in range(3647):
    if l0[i][0]== "live":
            print(i)

596


In [68]:

for i in range(575, 617):
              print(l0[i])

('number', 0.00027488408)
('javascript', 0.0002748771)
('common', 0.0002748651)
('year', 0.00027486042)
('dream', 0.00027485215)
('disguise', 0.0002748454)
('rhymes', 0.00027484496)
('company', 0.00027484)
('festival', 0.0002748358)
('opportunity', 0.00027483224)
('learned', 0.00027483108)
('further', 0.00027482855)
('explore', 0.00027482843)
('argument', 0.00027482645)
('studied', 0.00027482628)
('potential', 0.00027481467)
('dropped', 0.00027480492)
('piece', 0.0002748048)
('application', 0.00027480104)
('detail', 0.00027479386)
('dull', 0.0002747886)
('live', 0.00027478675)
('music', 0.0002747789)
('operations', 0.0002747677)
('title', 0.00027475727)
('beauty', 0.00027475547)
('implications', 0.00027475157)
('reviews', 0.00027474505)
('meant', 0.0002747419)
('pict', 0.0002747335)
('continue', 0.00027473186)
('2015', 0.0002747299)
('anticus', 0.0002747199)
('final', 0.00027470745)
('quotes', 0.00027470326)
('rhythms', 0.0002746988)
('authentic', 0.0002746918)
('xxviii', 0.00027467773

In [71]:

for i in range(3647):
    if l0[i][0]== "afternoons":
            print(i)

3579


In [72]:

for i in range(3647):
    if l1[i][0]== "afternoons":
            print(i)

1283


In [73]:

for i in range(3558, 3600):
              print(l0[i])

('sketch', 0.00027297795)
('risk', 0.00027297734)
('rhymingverse', 0.00027297676)
('captured', 0.00027297635)
('seeds', 0.0002729762)
('rană', 0.0002729755)
('controller', 0.00027297542)
('speechtotext', 0.00027297504)
('thor', 0.00027297303)
('circle', 0.00027297295)
('biggest', 0.00027297286)
('seizingthemedia', 0.00027297277)
('easytofollow', 0.0002729723)
('email', 0.00027297172)
('editorship', 0.00027297135)
('undercurrents', 0.0002729697)
('signifying', 0.00027296957)
('analogous', 0.00027296934)
('doctoral', 0.00027296826)
('telephone', 0.0002729676)
('bottom', 0.0002729674)
('afternoons', 0.00027296692)
('unwilling', 0.0002729669)
('indecisions', 0.00027296628)
('lashing', 0.00027296573)
('clock', 0.00027296506)
('același', 0.0002729649)
('lacking—', 0.00027296488)
('oven', 0.00027296186)
('fitter', 0.00027295764)
('cultural', 0.000272956)
('republic', 0.00027295586)
('charm', 0.00027295476)
('exist', 0.00027295412)
('șiastfel', 0.0002729536)
('rebuilt', 0.00027295333)
('asc

In [74]:

for i in range(1262, 1304):
              print(l1[i])

('sculpture', 0.00015720294)
('drowned', 0.0001572028)
('bestow', 0.00015720278)
('greatness', 0.00015720274)
('timeconsuming', 0.00015720262)
('piatră', 0.00015720256)
('ascii', 0.00015720249)
('rebuilt', 0.00015720248)
('șiastfel', 0.00015720245)
('exist', 0.00015720233)
('charm', 0.00015720223)
('republic', 0.00015720195)
('cultural', 0.00015720195)
('fitter', 0.0001572016)
('oven', 0.00015720079)
('același', 0.00015720024)
('lacking—', 0.00015720024)
('clock', 0.00015720016)
('lashing', 0.00015720006)
('indecisions', 0.00015719995)
('unwilling', 0.00015719983)
('afternoons', 0.00015719981)
('bottom', 0.00015719973)
('telephone', 0.0001571997)
('doctoral', 0.00015719954)
('analogous', 0.00015719933)
('signifying', 0.00015719929)
('undercurrents', 0.00015719929)
('editorship', 0.00015719894)
('email', 0.00015719885)
('easytofollow', 0.00015719878)
('seizingthemedia', 0.00015719866)
('biggest', 0.00015719865)
('circle', 0.00015719864)
('thor', 0.00015719862)
('speechtotext', 0.0001

In [75]:

for i in range(3647):
    if l0[i][0]== "couplets":
            print(i)

2780


In [76]:
for i in range(3647):
    if l1[i][0]== "couplets":
            print(i)

2082


In [77]:

for i in range(2759, 2801):
              print(l0[i])

('respectivei', 0.0002731354)
('angst', 0.00027313532)
('sick', 0.00027313532)
('difficulties', 0.00027313465)
('strongest', 0.0002731345)
('mechanical', 0.00027313438)
('contextualized', 0.0002731342)
('authors', 0.000273134)
('largescaleprojects', 0.00027313398)
('earnest', 0.00027313398)
('styles', 0.00027313386)
('enormousness', 0.00027313357)
('activities', 0.0002731335)
('aerului', 0.0002731334)
('worldwideweb', 0.00027313334)
('prescient', 0.0002731333)
('generating', 0.00027313322)
('preservation', 0.00027313308)
('roundabout', 0.00027313296)
('news', 0.00027313293)
('confoundered', 0.0002731328)
('couplets', 0.00027313278)
('reason', 0.00027313232)
('writhing', 0.00027313206)
('climates', 0.00027313203)
('illuminates', 0.00027313203)
('suffered', 0.000273132)
('landscapes', 0.00027313194)
('partnering', 0.00027313182)
('dabbler', 0.0002731318)
('liberties', 0.00027313177)
('lofi', 0.00027313156)
('scholarlywork', 0.00027313156)
('frustrations', 0.00027313136)
('sharingmoving',

In [78]:

for i in range(2061, 2103):
              print(l1[i])

('strange', 0.00015716812)
('trebuie', 0.0001571681)
('usefully', 0.0001571681)
('decision', 0.00015716809)
('thin', 0.00015716809)
('cofounded', 0.00015716808)
('meal', 0.00015716797)
('stipulated', 0.00015716792)
('sharingmoving', 0.0001571679)
('lofi', 0.00015716783)
('frustrations', 0.00015716783)
('scholarlywork', 0.00015716781)
('liberties', 0.00015716779)
('dabbler', 0.00015716779)
('partnering', 0.00015716777)
('landscapes', 0.00015716774)
('illuminates', 0.00015716774)
('writhing', 0.00015716774)
('climates', 0.00015716773)
('suffered', 0.00015716771)
('reason', 0.00015716764)
('couplets', 0.0001571676)
('confoundered', 0.00015716757)
('news', 0.00015716755)
('roundabout', 0.00015716754)
('generating', 0.00015716752)
('worldwideweb', 0.0001571675)
('preservation', 0.0001571675)
('aerului', 0.00015716748)
('activities', 0.00015716747)
('prescient', 0.00015716744)
('enormousness', 0.00015716742)
('authors', 0.00015716735)
('earnest', 0.00015716735)
('styles', 0.00015716732)
('la

In [24]:

len(l1)

3648

In [25]:

for i in range(3649):
    if l1[i][0]== "orderoftheday":
            print(i)

1577


IndexError: list index out of range

In [28]:

for i in range(3648):
    if l0[i][0]== "orderoftheday":
            print(i)

3286


In [29]:

for i in range(3265, 3307):
              print(l0[i])

('photography', 0.00015712235)
('seeger', 0.00015712234)
('date', 0.00015712228)
('moral', 0.00015712224)
('broaderaudience', 0.0001571222)
('technicalwork', 0.00015712218)
('debate—between', 0.00015712218)
('shelf', 0.00015712215)
('enhance', 0.00015712215)
('article', 0.00015712212)
('—drinks', 0.00015712205)
('laptop', 0.00015712202)
('fusty', 0.000157122)
('embraced', 0.00015712199)
('electronicpoetry', 0.00015712196)
('immersion', 0.00015712195)
('fitter', 0.00015712195)
('kept', 0.00015712192)
('weaker', 0.00015712187)
('programul', 0.0001571218)
('călătorie', 0.0001571218)
('orderoftheday', 0.00015712177)
('artful', 0.00015712177)
('frozen', 0.00015712177)
('selfless', 0.0001571217)
('sparte', 0.0001571217)
('donbyrd', 0.0001571217)
('hours', 0.0001571217)
('drill', 0.00015712167)
('analyze', 0.00015712167)
('sharingmoving', 0.00015712158)
('criptă', 0.00015712154)
('frustrations', 0.00015712154)
('television', 0.00015712148)
('cofounded', 0.00015712148)
('involving', 0.00015

In [30]:

for i in range(1556, 1598):
              print(l1[i])

('subjugated', 0.00027364772)
('traducere', 0.00027364772)
('considering', 0.00027364743)
('propel', 0.0002736474)
('exists', 0.00027364714)
('splendid', 0.0002736469)
('employed', 0.0002736467)
('involving', 0.00027364635)
('cofounded', 0.00027364623)
('television', 0.00027364618)
('criptă', 0.00027364594)
('frustrations', 0.00027364586)
('sharingmoving', 0.00027364568)
('analyze', 0.00027364527)
('hours', 0.00027364524)
('drill', 0.0002736452)
('donbyrd', 0.00027364513)
('selfless', 0.000273645)
('sparte', 0.00027364495)
('artful', 0.00027364478)
('frozen', 0.00027364478)
('orderoftheday', 0.0002736447)
('programul', 0.00027364443)
('călătorie', 0.00027364443)
('weaker', 0.0002736442)
('kept', 0.00027364393)
('immersion', 0.00027364393)
('electronicpoetry', 0.00027364376)
('fitter', 0.00027364376)
('embraced', 0.00027364373)
('fusty', 0.00027364338)
('laptop', 0.00027364332)
('—drinks', 0.00027364315)
('debate—between', 0.0002736427)
('technicalwork', 0.0002736427)
('enhance', 0.0

In [31]:

for i in range(3648):
    if l0[i][0]== "rules":
            print(i)

1608


In [32]:
for i in range(3648):
    if l1[i][0]== "rules":
            print(i)

3255


In [33]:

for i in range(3234, 3276):
              print(l1[i])

('life—', 0.00027351012)
('reflecting', 0.0002735101)
('metropolitană', 0.00027350994)
('hide', 0.00027350982)
('drown', 0.00027350974)
('liberties', 0.00027350968)
('succinctly', 0.00027350956)
('figures', 0.0002735095)
('fellow', 0.00027350945)
('illicit', 0.00027350936)
('manageable', 0.0002735093)
('originalele', 0.00027350924)
('engages', 0.0002735092)
('piecing', 0.00027350918)
('arrives', 0.0002735091)
('caring', 0.00027350895)
('latest', 0.00027350883)
('lysistrata', 0.0002735088)
('float', 0.0002735086)
('imagepersonandplace', 0.00027350854)
('welcomed', 0.0002735081)
('rules', 0.00027350808)
('catchsome', 0.000273508)
('megacrash', 0.0002735079)
('adventures', 0.0002735077)
('dark', 0.00027350755)
('diminish', 0.00027350738)
('hover', 0.00027350732)
('largeonlineworks', 0.0002735073)
('xliii', 0.00027350726)
('kick', 0.0002735072)
('brian', 0.00027350718)
('victorian', 0.00027350677)
('serving', 0.0002735067)
('lacks', 0.00027350665)
('culture', 0.00027350656)
('brilliant', 

In [34]:

for i in range(1587, 1629):
              print(l0[i])

('writer', 0.00015714871)
('hufnagel', 0.00015714864)
('noon', 0.00015714859)
('acrosstheworld', 0.00015714859)
('spaces', 0.00015714859)
('import', 0.00015714858)
('brilliant', 0.00015714856)
('lacks', 0.00015714852)
('culture', 0.00015714852)
('serving', 0.00015714852)
('victorian', 0.0001571485)
('brian', 0.00015714849)
('hover', 0.00015714843)
('largeonlineworks', 0.00015714842)
('xliii', 0.00015714842)
('kick', 0.0001571484)
('dark', 0.00015714839)
('diminish', 0.00015714837)
('adventures', 0.00015714833)
('megacrash', 0.00015714829)
('catchsome', 0.00015714829)
('rules', 0.00015714829)
('welcomed', 0.00015714827)
('imagepersonandplace', 0.00015714818)
('float', 0.00015714816)
('lysistrata', 0.00015714814)
('latest', 0.00015714811)
('caring', 0.00015714808)
('arrives', 0.00015714808)
('engages', 0.00015714807)
('piecing', 0.00015714807)
('originalele', 0.00015714807)
('illicit', 0.00015714802)
('manageable', 0.000157148)
('fellow', 0.000157148)
('figures', 0.00015714798)
('succinc

In [35]:

for i in range(3648):
    if l0[i][0]== "adolescent":
            print(i)

789


In [36]:
for i in range(3648):
    if l1[i][0]== "adolescent":
            print(i)

1091


In [37]:

for i in range(768, 810):
              print(l0[i])

('connect', 0.0002619622)
('generator', 0.00026196215)
('talk', 0.0002619621)
('dawning', 0.00026196204)
('yours', 0.00026196198)
('mixed', 0.00026196195)
('return', 0.00026196192)
('hypermedia', 0.00026196166)
('shattered', 0.0002619616)
('case', 0.00026196116)
('fifty', 0.00026196073)
('esque', 0.00026196073)
('beaten', 0.0002619605)
('links', 0.00026196046)
('potentially', 0.00026196046)
('uncovered', 0.00026196038)
('casting', 0.00026196014)
('expertise', 0.00026196014)
('nota', 0.00026196011)
('split', 0.00026196003)
('elizabethan', 0.00026195974)
('adolescent', 0.0002619593)
('tape', 0.00026195913)
('body', 0.0002619591)
('details', 0.00026195895)
('minutes', 0.00026195892)
('deep', 0.0002619588)
('funded', 0.0002619586)
('1977', 0.00026195805)
('invites', 0.0002619579)
('earlier', 0.00026195784)
('crypt', 0.00026195752)
('pentameter', 0.00026195744)
('insight', 0.0002619573)
('playback', 0.00026195723)
('bass', 0.0002619571)
('gone', 0.0002619571)
('layered', 0.00026195656)
('oc

In [38]:

for i in range(1070, 1112):
              print(l1[i])

('fools', 0.0002739789)
('discuss', 0.0002739787)
('college', 0.00027397813)
('curiosity', 0.00027397738)
('occurred', 0.00027397723)
('layered', 0.00027397668)
('bass', 0.0002739742)
('gone', 0.00027397386)
('playback', 0.0002739728)
('insight', 0.00027397278)
('pentameter', 0.0002739722)
('crypt', 0.00027397167)
('invites', 0.00027396978)
('earlier', 0.00027396966)
('1977', 0.00027396885)
('funded', 0.00027396626)
('deep', 0.00027396512)
('minutes', 0.0002739644)
('details', 0.0002739642)
('body', 0.0002739635)
('tape', 0.0002739634)
('adolescent', 0.00027396285)
('elizabethan', 0.0002739606)
('split', 0.0002739586)
('expertise', 0.0002739582)
('nota', 0.00027395817)
('casting', 0.0002739579)
('uncovered', 0.0002739569)
('beaten', 0.00027395657)
('links', 0.00027395645)
('potentially', 0.00027395628)
('fifty', 0.00027395508)
('esque', 0.00027395497)
('case', 0.000273953)
('shattered', 0.00027395017)
('hypermedia', 0.00027395017)
('return', 0.00027394915)
('yours', 0.0002739487)
('mix

In [39]:

for i in range(3648):
    if l0[i][0]== "urbane":
            print(i)

3491


In [40]:
for i in range(3648):
    if l1[i][0]== "urbane":
            print(i)

1372


In [41]:

for i in range(3470, 3512):
              print(l0[i])

('mode', 0.00015711588)
('high', 0.00015711588)
('time—of', 0.00015711588)
('bucurii', 0.00015711572)
('brazen', 0.00015711572)
('midiazine', 0.00015711562)
('forgotten', 0.00015711562)
('locations', 0.0001571156)
('xxxvi', 0.00015711554)
('ejournal', 0.00015711541)
('primitive', 0.00015711534)
('njit', 0.00015711534)
('channel', 0.00015711534)
('band', 0.00015711528)
('dabbler', 0.00015711527)
('obfuscation', 0.00015711524)
('before—', 0.00015711522)
('approximately', 0.0001571152)
('lovers', 0.00015711512)
('rushed', 0.00015711511)
('primary', 0.0001571151)
('urbane', 0.00015711506)
('machines', 0.00015711505)
('cent', 0.00015711504)
('progressed', 0.00015711499)
('rebirth—', 0.00015711493)
('caution', 0.00015711492)
('throats', 0.00015711486)
('muffle', 0.0001571148)
('sounddesign', 0.00015711477)
('splashes', 0.00015711476)
('babies', 0.00015711473)
('prizat', 0.00015711471)
('freedom', 0.0001571147)
('players', 0.00015711453)
('constantatomis', 0.00015711447)
('obsession', 0.00015

In [42]:

for i in range(1351, 1392):
              print(l1[i])

('ascii', 0.000273684)
('wikipedia', 0.00027368325)
('phoneme', 0.00027368293)
('download', 0.00027368288)
('searched', 0.00027368258)
('circuite', 0.00027368253)
('constantatomis', 0.00027368244)
('obsession', 0.00027368235)
('players', 0.00027368192)
('freedom', 0.0002736813)
('prizat', 0.00027368127)
('babies', 0.000273681)
('splashes', 0.00027368087)
('sounddesign', 0.00027368075)
('muffle', 0.00027368055)
('throats', 0.0002736802)
('caution', 0.00027368014)
('rebirth—', 0.00027367994)
('progressed', 0.00027367967)
('cent', 0.00027367944)
('machines', 0.0002736793)
('urbane', 0.00027367927)
('lovers', 0.00027367906)
('primary', 0.000273679)
('rushed', 0.00027367895)
('approximately', 0.00027367863)
('before—', 0.0002736783)
('band', 0.00027367828)
('dabbler', 0.00027367816)
('obfuscation', 0.00027367804)
('channel', 0.0002736779)
('primitive', 0.0002736779)
('njit', 0.00027367784)
('ejournal', 0.00027367737)
('xxxvi', 0.0002736768)
('forgotten', 0.00027367644)
('locations', 0.00027

In [44]:

for i in range(3648):
    if l0[i][0]== "outside":
            print(i)

3313


In [45]:

for i in range(3648):
    if l1[i][0]== "outside":
            print(i)

1551


In [46]:

for i in range(3292, 3334):
              print(l0[i])

('hours', 0.0001571217)
('drill', 0.00015712167)
('analyze', 0.00015712167)
('sharingmoving', 0.00015712158)
('criptă', 0.00015712154)
('frustrations', 0.00015712154)
('television', 0.00015712148)
('cofounded', 0.00015712148)
('involving', 0.00015712145)
('employed', 0.0001571214)
('splendid', 0.00015712138)
('exists', 0.00015712132)
('propel', 0.00015712126)
('considering', 0.00015712122)
('subjugated', 0.0001571212)
('confound', 0.00015712119)
('traducere', 0.00015712118)
('shock—', 0.00015712115)
('variables', 0.00015712115)
('idle', 0.0001571211)
('cotitură', 0.00015712103)
('outside', 0.000157121)
('penchant', 0.000157121)
('leonardo', 0.00015712097)
('crash', 0.00015712094)
('california', 0.00015712094)
('mixable', 0.00015712094)
('artillerists', 0.00015712091)
('egoism', 0.00015712091)
('hypertext', 0.00015712086)
('calypso', 0.00015712083)
('found—and', 0.00015712083)
('rent', 0.0001571208)
('19591995', 0.00015712077)
('compared', 0.00015712075)
('twentyfour', 0.00015712075)


In [47]:

for i in range(1530, 1572):
              print(l1[i])

('anglosaxoni', 0.00027365104)
('predetermined', 0.00027365095)
('reflect', 0.00027365083)
('randomly', 0.00027365077)
('solace', 0.00027365074)
('party', 0.0002736503)
('twentyfour', 0.00027365002)
('compared', 0.00027365)
('19591995', 0.0002736499)
('rent', 0.00027364973)
('calypso', 0.00027364958)
('found—and', 0.00027364958)
('hypertext', 0.00027364938)
('artillerists', 0.00027364917)
('crash', 0.00027364914)
('egoism', 0.00027364914)
('mixable', 0.0002736491)
('leonardo', 0.0002736489)
('california', 0.0002736489)
('penchant', 0.00027364865)
('cotitură', 0.00027364862)
('outside', 0.00027364856)
('idle', 0.0002736483)
('shock—', 0.00027364807)
('variables', 0.00027364792)
('confound', 0.00027364783)
('subjugated', 0.00027364772)
('traducere', 0.00027364772)
('considering', 0.00027364743)
('propel', 0.0002736474)
('exists', 0.00027364714)
('splendid', 0.0002736469)
('employed', 0.0002736467)
('involving', 0.00027364635)
('cofounded', 0.00027364623)
('television', 0.00027364618)
('

In [49]:

for i in range(3648):
    if l0[i][0]== "constantatomis":
            print(i)

3505


In [50]:

for i in range(3648):
    if l1[i][0]== "constantatomis":
            print(i)

1357


In [51]:

for i in range(3484, 3526):
              print(l0[i])

('dabbler', 0.00015711527)
('obfuscation', 0.00015711524)
('before—', 0.00015711522)
('approximately', 0.0001571152)
('lovers', 0.00015711512)
('rushed', 0.00015711511)
('primary', 0.0001571151)
('urbane', 0.00015711506)
('machines', 0.00015711505)
('cent', 0.00015711504)
('progressed', 0.00015711499)
('rebirth—', 0.00015711493)
('caution', 0.00015711492)
('throats', 0.00015711486)
('muffle', 0.0001571148)
('sounddesign', 0.00015711477)
('splashes', 0.00015711476)
('babies', 0.00015711473)
('prizat', 0.00015711471)
('freedom', 0.0001571147)
('players', 0.00015711453)
('constantatomis', 0.00015711447)
('obsession', 0.00015711445)
('searched', 0.00015711444)
('circuite', 0.0001571144)
('download', 0.00015711437)
('phoneme', 0.00015711437)
('wikipedia', 0.0001571143)
('legroom', 0.00015711412)
('ascii', 0.0001571141)
('sixteen', 0.000157114)
('tendencies', 0.00015711391)
('hear', 0.0001571139)
('scribblings', 0.00015711389)
('timeconsuming', 0.00015711381)
('dries', 0.00015711381)
('strat

In [52]:

for i in range(1336, 1378):
              print(l1[i])

('foundations', 0.00027368768)
('humorous', 0.00027368765)
('nouveau', 0.0002736874)
('nuanța', 0.00027368678)
('explorable', 0.00027368654)
('rings', 0.00027368614)
('stratiforms', 0.00027368605)
('dries', 0.0002736859)
('timeconsuming', 0.0002736856)
('apoi', 0.00027368558)
('hear', 0.00027368538)
('scribblings', 0.00027368532)
('tendencies', 0.00027368517)
('sixteen', 0.0002736846)
('legroom', 0.00027368404)
('ascii', 0.000273684)
('wikipedia', 0.00027368325)
('phoneme', 0.00027368293)
('download', 0.00027368288)
('searched', 0.00027368258)
('circuite', 0.00027368253)
('constantatomis', 0.00027368244)
('obsession', 0.00027368235)
('players', 0.00027368192)
('freedom', 0.0002736813)
('prizat', 0.00027368127)
('babies', 0.000273681)
('splashes', 0.00027368087)
('sounddesign', 0.00027368075)
('muffle', 0.00027368055)
('throats', 0.0002736802)
('caution', 0.00027368014)
('rebirth—', 0.00027367994)
('progressed', 0.00027367967)
('cent', 0.00027367944)
('machines', 0.0002736793)
('urbane

In [54]:

for i in range(3648):
    if l0[i][0]== "trysts":
            print(i)

1738


In [55]:

for i in range(3648):
    if l1[i][0]== "trysts":
            print(i)

3128


In [60]:

for i in range(1717, 1759):
              print(l0[i])

('gleaming', 0.0001571462)
('interconnection', 0.0001571462)
('anguished', 0.0001571462)
('explained', 0.00015714619)
('combining', 0.00015714619)
('minuscule', 0.00015714616)
('regard', 0.00015714612)
('knife', 0.00015714612)
('tamburlaine', 0.0001571461)
('nuanced', 0.0001571461)
('rochester', 0.00015714609)
('viable', 0.00015714607)
('wealth', 0.00015714603)
('returns', 0.00015714603)
('climax', 0.00015714603)
('webbasedliterary', 0.00015714602)
('—protagonist', 0.00015714602)
('suntem', 0.00015714593)
('sources', 0.00015714591)
('gestalt', 0.00015714591)
('zeubogăție', 0.0001571459)
('trysts', 0.0001571459)
('surgicalfileedits', 0.0001571459)
('accompanied', 0.00015714586)
('collectively', 0.00015714586)
('crisis', 0.00015714586)
('formatting', 0.00015714584)
('poeticdiscourse', 0.00015714584)
('procedură', 0.00015714584)
('words—are', 0.00015714583)
('reciter', 0.00015714583)
('what—a', 0.00015714583)
('selfmotivated', 0.00015714578)
('hell', 0.00015714578)
('wordofmouth', 0.00

In [61]:

for i in range(3097, 3149):
              print(l1[i])

('effective', 0.00027352234)
('codingtechniques', 0.0002735223)
('hefty', 0.00027352228)
('thirst', 0.00027352222)
('nominated', 0.00027352193)
('poetryandaudiopractice', 0.00027352181)
('deliverysystems', 0.00027352181)
('attacked', 0.00027352164)
('proper', 0.00027352164)
('equaling', 0.0002735216)
('erau', 0.00027352155)
('succeeded', 0.00027352152)
('remediation', 0.0002735214)
('mask', 0.00027352123)
('selfindulgence', 0.0002735212)
('wordofmouth', 0.00027352117)
('hell', 0.00027352094)
('selfmotivated', 0.00027352083)
('words—are', 0.00027352074)
('crisis', 0.0002735207)
('procedură', 0.0002735207)
('reciter', 0.00027352068)
('poeticdiscourse', 0.00027352062)
('what—a', 0.00027352048)
('accompanied', 0.00027352048)
('formatting', 0.00027352045)
('collectively', 0.00027352042)
('surgicalfileedits', 0.0002735204)
('sources', 0.00027352036)
('gestalt', 0.00027352027)
('zeubogăție', 0.00027352027)
('trysts', 0.00027352013)
('suntem', 0.00027352013)
('—protagonist', 0.00027351998)


In [56]:

for i in range(3648):
    if l0[i][0]== "illicit":
            print(i)

1619


In [57]:

for i in range(3648):
    if l1[i][0]== "illicit":
            print(i)

3243


In [62]:

for i in range(3648):
    if l0[i][0]== "impotence":
            print(i)

1282


In [59]:

for i in range(3648):
    if l1[i][0]== "impotence":
            print(i)

3581


In [64]:

for i in range(1261, 1303):
              print(l0[i])

('pole', 0.00015716127)
('platforme', 0.00015716121)
('sophisticated', 0.00015716121)
('expressiveaudio', 0.00015716118)
('credit', 0.00015716115)
('mystify', 0.00015716105)
('spoken—', 0.00015716096)
('hands', 0.00015716093)
('goin', 0.00015716092)
('zin', 0.00015716073)
('midtide', 0.00015716068)
('living', 0.00015716063)
('cruel', 0.0001571606)
('checking', 0.00015716052)
('drawn', 0.00015716036)
('saucy', 0.00015716012)
('maligned', 0.00015716004)
('risking', 0.00015715964)
('anumite', 0.0001571596)
('academicstudy', 0.00015715953)
('italicized', 0.00015715952)
('impotence', 0.00015715948)
('instructed', 0.0001571591)
('seas', 0.00015715908)
('scan', 0.00015715907)
('excerpt', 0.00015715891)
('facetime', 0.00015715891)
('foto', 0.00015715887)
('overlooked—', 0.00015715862)
('goes', 0.00015715862)
('satisfaction', 0.0001571585)
('concerned', 0.00015715843)
('woke', 0.0001571584)
('believes', 0.00015715836)
('experimental', 0.00015715831)
('transparent—surely', 0.00015715825)
('resea

In [63]:

for i in range(3560, 3602):
              print(l1[i])

('construit', 0.00027345866)
('xxxviii', 0.00027345808)
('instantaneous', 0.0002734576)
('thor', 0.0002734573)
('referred', 0.0002734572)
('researchmechanism', 0.00027345706)
('attacking', 0.000273457)
('transparent—surely', 0.0002734564)
('experimental', 0.00027345624)
('woke', 0.00027345584)
('believes', 0.00027345578)
('concerned', 0.0002734557)
('satisfaction', 0.00027345523)
('overlooked—', 0.00027345459)
('goes', 0.00027345453)
('foto', 0.00027345357)
('facetime', 0.00027345313)
('excerpt', 0.000273453)
('scan', 0.00027345237)
('instructed', 0.0002734521)
('seas', 0.00027345208)
('impotence', 0.0002734503)
('italicized', 0.0002734501)
('academicstudy', 0.00027345)
('anumite', 0.00027344972)
('risking', 0.00027344926)
('maligned', 0.00027344737)
('saucy', 0.0002734468)
('drawn', 0.00027344574)
('checking', 0.0002734449)
('living', 0.00027344434)
('cruel', 0.00027344434)
('midtide', 0.00027344408)
('zin', 0.00027344387)
('goin', 0.00027344289)
('hands', 0.00027344283)
('spoken—', 0

In [66]:

for i in range(3648):
    if l0[i][0]== "adventures":
            print(i)

1605


In [67]:

for i in range(3648):
    if l1[i][0]== "adventures":
            print(i)

3258


In [69]:

for i in range(1584, 1635):
              print(l0[i])

('fluctuation', 0.00015714878)
('annotated', 0.00015714875)
('offered', 0.00015714871)
('writer', 0.00015714871)
('hufnagel', 0.00015714864)
('noon', 0.00015714859)
('acrosstheworld', 0.00015714859)
('spaces', 0.00015714859)
('import', 0.00015714858)
('brilliant', 0.00015714856)
('lacks', 0.00015714852)
('culture', 0.00015714852)
('serving', 0.00015714852)
('victorian', 0.0001571485)
('brian', 0.00015714849)
('hover', 0.00015714843)
('largeonlineworks', 0.00015714842)
('xliii', 0.00015714842)
('kick', 0.0001571484)
('dark', 0.00015714839)
('diminish', 0.00015714837)
('adventures', 0.00015714833)
('megacrash', 0.00015714829)
('catchsome', 0.00015714829)
('rules', 0.00015714829)
('welcomed', 0.00015714827)
('imagepersonandplace', 0.00015714818)
('float', 0.00015714816)
('lysistrata', 0.00015714814)
('latest', 0.00015714811)
('caring', 0.00015714808)
('arrives', 0.00015714808)
('engages', 0.00015714807)
('piecing', 0.00015714807)
('originalele', 0.00015714807)
('illicit', 0.00015714802)
(

In [70]:

for i in range(3237, 3279):
              print(l1[i])

('hide', 0.00027350982)
('drown', 0.00027350974)
('liberties', 0.00027350968)
('succinctly', 0.00027350956)
('figures', 0.0002735095)
('fellow', 0.00027350945)
('illicit', 0.00027350936)
('manageable', 0.0002735093)
('originalele', 0.00027350924)
('engages', 0.0002735092)
('piecing', 0.00027350918)
('arrives', 0.0002735091)
('caring', 0.00027350895)
('latest', 0.00027350883)
('lysistrata', 0.0002735088)
('float', 0.0002735086)
('imagepersonandplace', 0.00027350854)
('welcomed', 0.0002735081)
('rules', 0.00027350808)
('catchsome', 0.000273508)
('megacrash', 0.0002735079)
('adventures', 0.0002735077)
('dark', 0.00027350755)
('diminish', 0.00027350738)
('hover', 0.00027350732)
('largeonlineworks', 0.0002735073)
('xliii', 0.00027350726)
('kick', 0.0002735072)
('brian', 0.00027350718)
('victorian', 0.00027350677)
('serving', 0.0002735067)
('lacks', 0.00027350665)
('culture', 0.00027350656)
('brilliant', 0.0002735065)
('import', 0.00027350645)
('noon', 0.0002735064)
('spaces', 0.00027350633)

In [71]:

for i in range(3648):
    if l0[i][0]== "danger":
            print(i)

2801


In [72]:

for i in range(3648):
    if l1[i][0]== "danger":
            print(i)

2063


In [73]:

for i in range(2780, 2822):
              print(l0[i])

('consisting', 0.00015713078)
('institution', 0.00015713075)
('dryup', 0.00015713075)
('immense', 0.00015713075)
('historically', 0.00015713075)
('screen', 0.00015713074)
('setup', 0.00015713074)
('catalog', 0.00015713072)
('bypassing', 0.0001571307)
('continually', 0.0001571307)
('william', 0.0001571307)
('amateur', 0.0001571307)
('sore', 0.0001571307)
('connections', 0.00015713068)
('nonsynchronous', 0.00015713068)
('înghețate', 0.00015713068)
('series', 0.00015713066)
('temporarily', 0.00015713066)
('știam', 0.00015713065)
('halocare', 0.00015713062)
('draw', 0.0001571306)
('danger', 0.00015713059)
('sketch', 0.00015713055)
('enwikipediaorgwikifactsheetfive', 0.00015713055)
('almost', 0.00015713052)
('audioeditions', 0.0001571305)
('bogat', 0.0001571305)
('arranging', 0.00015713049)
('preservation', 0.00015713047)
('wishing', 0.00015713046)
('founded', 0.00015713046)
('translate', 0.00015713045)
('suprafața', 0.00015713045)
('sifting', 0.00015713042)
('mark', 0.0001571304)
('mea

In [74]:

for i in range(2042, 2084):
              print(l1[i])

('constrict', 0.00027360057)
('discovering', 0.00027360057)
('professional', 0.00027360054)
('underestimate', 0.0002736005)
('benefited', 0.0002736004)
('resistance', 0.00027360028)
('meagre', 0.00027360028)
('sifting', 0.00027360025)
('translate', 0.00027360002)
('mark', 0.00027360002)
('suprafața', 0.00027360002)
('founded', 0.0002736)
('wishing', 0.00027359993)
('audioeditions', 0.00027359984)
('preservation', 0.00027359984)
('arranging', 0.00027359976)
('almost', 0.00027359964)
('bogat', 0.0002735996)
('sketch', 0.00027359955)
('enwikipediaorgwikifactsheetfive', 0.0002735995)
('draw', 0.00027359932)
('danger', 0.0002735993)
('halocare', 0.00027359917)
('connections', 0.00027359888)
('temporarily', 0.00027359888)
('nonsynchronous', 0.00027359882)
('continually', 0.0002735988)
('înghețate', 0.00027359877)
('bypassing', 0.00027359877)
('amateur', 0.00027359877)
('series', 0.00027359874)
('william', 0.0002735987)
('știam', 0.0002735987)
('screen', 0.00027359865)
('catalog', 0.00027

In [75]:

for i in range(3648):
    if l0[i][0]== "intrigue":
            print(i)

962


In [76]:

for i in range(941, 983):
              print(l0[i])

('local', 0.00026193995)
('broadly', 0.0002619399)
('acts', 0.00026193986)
('control', 0.0002619395)
('inbetween', 0.0002619395)
('waves', 0.00026193945)
('refine', 0.0002619394)
('irrelevant', 0.00026193936)
('extra', 0.00026193936)
('xxix', 0.00026193928)
('mashtalir', 0.00026193916)
('components', 0.00026193875)
('epic', 0.00026193834)
('2010', 0.00026193834)
('directly', 0.00026193826)
('easy', 0.00026193794)
('deptford', 0.0002619379)
('designed', 0.00026193773)
('stuck', 0.00026193724)
('false', 0.0002619371)
('stronger', 0.00026193704)
('intrigue', 0.00026193695)
('analog', 0.00026193683)
('mechanism', 0.00026193683)
('active', 0.00026193674)
('hand', 0.00026193654)
('influences', 0.00026193637)
('artworks', 0.0002619363)
('atât', 0.00026193578)
('blues', 0.00026193575)
('converter', 0.00026193558)
('crucial', 0.00026193543)
('prewww', 0.00026193523)
('covent', 0.00026193517)
('large', 0.00026193503)
('riffs', 0.000261935)
('microphone', 0.00026193494)
('real', 0.0002619348)
('

In [77]:

for i in range(3648):
    if l1[i][0]== "intrigue":
            print(i)

917


In [78]:

for i in range(896, 938):
              print(l1[i])

('bored', 0.00027409312)
('cassetterecorder', 0.00027409272)
('frith', 0.00027409132)
('knowing', 0.0002740895)
('real', 0.0002740894)
('riffs', 0.00027408855)
('microphone', 0.00027408823)
('large', 0.000274088)
('prewww', 0.00027408727)
('covent', 0.00027408722)
('crucial', 0.00027408596)
('converter', 0.00027408512)
('eventually', 0.00027408442)
('blues', 0.00027408425)
('atât', 0.00027408387)
('artworks', 0.00027408142)
('influences', 0.0002740811)
('hand', 0.0002740805)
('active', 0.00027407962)
('analog', 0.00027407895)
('mechanism', 0.00027407857)
('intrigue', 0.00027407822)
('stronger', 0.00027407779)
('false', 0.00027407726)
('stuck', 0.00027407683)
('designed', 0.0002740744)
('deptford', 0.00027407365)
('easy', 0.00027407316)
('directly', 0.00027407156)
('epic', 0.00027407098)
('2010', 0.00027407074)
('components', 0.000274069)
('berry', 0.000274069)
('mashtalir', 0.00027406696)
('xxix', 0.00027406611)
('refine', 0.00027406565)
('irrelevant', 0.00027406547)
('extra', 0.00027

In [81]:

for i in range(3648):
    if l0[i][0]== "density":
            print(i)

1404


In [82]:
for i in range(3648):
    if l1[i][0]== "density":
            print(i)

3459


In [83]:

for i in range(1383, 1425):
              print(l0[i])

('stories', 0.00015715437)
('coronavirus', 0.00015715435)
('fast', 0.00015715434)
('ethos', 0.00015715428)
('fields', 0.00015715428)
('hierarchies', 0.00015715427)
('movement', 0.0001571542)
('marlowe—edward', 0.0001571542)
('ravel', 0.0001571542)
('tourists', 0.00015715418)
('basketed', 0.0001571541)
('tragedies', 0.0001571541)
('anglosaxone', 0.00015715409)
('convertible', 0.00015715406)
('numerous', 0.00015715403)
('london—marlowe', 0.00015715403)
('support—', 0.00015715402)
('notably', 0.00015715396)
('resolve', 0.00015715393)
('subsaharan', 0.00015715385)
('keepup', 0.00015715377)
('density', 0.00015715376)
('emphasises', 0.00015715361)
('worldwide', 0.00015715357)
('listeningsession', 0.00015715357)
('feel', 0.00015715351)
('joined', 0.00015715348)
('fericirea', 0.00015715347)
('revised', 0.00015715347)
('sofa', 0.00015715344)
('acquired', 0.00015715336)
('activategenerate', 0.00015715335)
('drudge', 0.00015715334)
('hybridized', 0.0001571533)
('solved', 0.00015715329)
('occurren

In [84]:

for i in range(3438, 3480):
              print(l1[i])

('limits', 0.00027348334)
('metropolitan', 0.0002734833)
('anglosaxons', 0.00027348305)
('cunning', 0.00027348276)
('rebuilt', 0.00027348273)
('specifications', 0.0002734824)
('gave', 0.00027348235)
('solved', 0.0002734822)
('hybridized', 0.00027348206)
('occurrence', 0.00027348203)
('activategenerate', 0.00027348188)
('acquired', 0.00027348177)
('drudge', 0.00027348168)
('sofa', 0.0002734813)
('fericirea', 0.00027348124)
('joined', 0.0002734812)
('revised', 0.00027348116)
('feel', 0.00027348095)
('worldwide', 0.00027348078)
('listeningsession', 0.0002734807)
('emphasises', 0.00027348043)
('density', 0.00027347976)
('keepup', 0.0002734796)
('subsaharan', 0.0002734794)
('resolve', 0.00027347883)
('notably', 0.00027347868)
('support—', 0.00027347848)
('numerous', 0.00027347845)
('london—marlowe', 0.00027347833)
('convertible', 0.00027347813)
('tragedies', 0.0002734781)
('anglosaxone', 0.0002734781)
('basketed', 0.000273478)
('tourists', 0.00027347764)
('marlowe—edward', 0.0002734776)
('m

In [85]:

for i in range(3648):
    if l0[i][0]== "fluidity":
            print(i)

1531


In [86]:

for i in range(3648):
    if l1[i][0]== "fluidity":
            print(i)

3332


In [87]:

for i in range(3648):
    if l0[i][0]== "authentic":
            print(i)

669


In [88]:

for i in range(1510, 1552):
              print(l0[i])

('preparation', 0.00015715082)
('confusion', 0.00015715082)
('meal', 0.0001571508)
('switch', 0.0001571508)
('opposition', 0.00015715076)
('sell', 0.00015715072)
('219', 0.00015715072)
('provocative', 0.00015715066)
('african', 0.00015715064)
('inclined', 0.00015715064)
('infamous', 0.00015715061)
('bores', 0.00015715054)
('dynamic', 0.00015715047)
('etymology', 0.00015715047)
('networkinterfaces', 0.00015715044)
('resolution', 0.00015715044)
('bucharest', 0.00015715043)
('remain', 0.00015715041)
('goodbye', 0.00015715041)
('view—from', 0.00015715038)
('prehistoricdigitalpoetry', 0.00015715038)
('fluidity', 0.00015715037)
('ableton', 0.00015715034)
('frieze', 0.00015715027)
('april', 0.00015715027)
('husband', 0.00015715022)
('tagging', 0.00015715009)
('concentrate', 0.00015715006)
('woes', 0.00015714992)
('bridled', 0.00015714989)
('mobiletechnologies', 0.00015714986)
('narrator', 0.00015714984)
('balding', 0.00015714984)
('newarkreview', 0.00015714981)
('publichtml', 0.0001571498)
('

In [89]:

for i in range(3311, 3353):
              print(l1[i])

('endeavour', 0.00027350162)
('colosseum', 0.00027350147)
('rhymingverse', 0.00027350138)
('witless', 0.00027350127)
('nicely', 0.00027350074)
('upsides', 0.00027350045)
('instruction', 0.00027350034)
('middle', 0.0002735002)
('publichtml', 0.0002735001)
('newarkreview', 0.0002735001)
('narrator', 0.0002734999)
('mobiletechnologies', 0.00027349987)
('balding', 0.00027349978)
('bridled', 0.00027349975)
('woes', 0.00027349973)
('concentrate', 0.0002734989)
('tagging', 0.0002734988)
('husband', 0.00027349795)
('frieze', 0.00027349783)
('april', 0.00027349766)
('ableton', 0.00027349737)
('fluidity', 0.00027349725)
('view—from', 0.00027349713)
('bucharest', 0.0002734971)
('remain', 0.00027349702)
('prehistoricdigitalpoetry', 0.00027349693)
('networkinterfaces', 0.00027349687)
('goodbye', 0.00027349687)
('resolution', 0.00027349687)
('dynamic', 0.0002734968)
('etymology', 0.00027349673)
('bores', 0.00027349632)
('infamous', 0.00027349606)
('african', 0.00027349603)
('inclined', 0.00027349588